In [ ]:
PATHFunctions = raw"C:\Users\Siby\Dropbox\git-repos\All-Process";
push!( LOAD_PATH, PATHFunctions );

using F2023
using JLD

FILEBRW = raw"C:\Users\Siby\Desktop\Datos\Hipocampo\BRWs\Phase_10.brw";
FILEVARS = FindContent( "Variables.jld", FILEBRW );
Variables = LoadDict( FILEVARS );

FILESTIMES = FindContent( "times", FILEBRW );

n = 4043;
FILETIMES = FILESTIMES[ n ];

K = collect( keys( LoadDict( FILETIMES ) ) );

aux = LoadDict( FILETIMES );

aux[ "spikes" ]
aux[ "cluster_class" ]

aux[ "par/sr"]

In [ ]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
limupper = 0.6; # GB maximum size of each segment considering Float16 format
ΔT = 250; # ms para el ΔV
sigma = 3; # para la convolucion gaussiana
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Constants = Dict(
    # for detecting saturations
    "maxvolt" => 3000, # μV
    "minvolt" => -3000, # μV
    # for the neighborhood
    "maxrange" => 500, # μV 
    "minrange" => -1000, # μV
    "minchannels" => 4, # valid channels on the neighborhood
    "maxradius" => 3, # maximus radious for searching the neighborhood
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
using JLD
using Plots
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@suppress begin
    PATHFunctions = 
    "/home/isabel/Dropbox/git-repos/All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILEBRW = "/run/media/isabel/Data/Hipocampo/Espontanea/01-06-2022/BRWs/Phase_10.brw";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@time Variables, FILEPATHS, FILEVARS = GetVarsHDF5( FILEBRW );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
σ = ChunkSizeSpace( Variables, limupper );
n0s = length( string( σ ) );
nChs = Variables[ "nChs" ];
channels = collect( 1:nChs );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS = load( FILEPATHS )[ "PATHS" ];
PATHMAIN = PATHS[ "PATHMAIN" ];
PATHFIGURES = joinpath( PATHMAIN, "Figures" ); mkpath( PATHFIGURES );
PATHFIGS_TEMP = joinpath( PATHFIGURES, "Step0" ); mkpath( PATHFIGS_TEMP );
PATHVOLTAGE = joinpath( PATHMAIN, "Voltage" ); mkpath( PATHVOLTAGE );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS[ "PATHFIGURES" ] = PATHFIGURES;
PATHS[ "PATHVOLTAGE" ] = PATHVOLTAGE;
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
save( FILEPATHS, "PATHS", PATHS );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@time for n = 23:σ
    BINRAW = OneSegment( Variables, n, σ ); # bin in uInt format
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW = Digital2Analogue( Variables, BINRAW ); # bin in μV
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINNAME = string( "BIN", lpad( n, n0s, "0" ), ".jld" );
    FILEBIN = joinpath( PATHVOLTAGE, BINNAME );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    # Pre eliminacion de los extremos
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    σΔV = stdΔV( Variables, BINRAW, ΔT );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    RowCount = UniqueCount( BINRAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    data = RemoveInfs( log.( σΔV ) ); 
    P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
    h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
    data = RowCount; 
    P3 = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); P3 = title!( "# V values" )
    h = copy( RowCount ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
    PF1 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    Discarded, BINRAW = RemoveExtrema( Variables, Constants, BINRAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    # Post eliminacion de los extremos
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    σΔV = stdΔV( Variables, BINRAW, ΔT );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    RowCount = UniqueCount( BINRAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    data = RemoveInfs( log.( σΔV ) ); 
    P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
    h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
    data = RowCount; 
    P3 = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); P3 = title!( "# V values" )
    h = copy( RowCount ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
    PF2 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    FFS0 = plot( PF1, PF2, layout = ( 2, 1 ) );
    FIGNAME = joinpath( PATHFIGS_TEMP, BINNAME );
    FIGNAME = replace( FIGNAME, ".jld" => ".svg" );
    savefig( FFS0, FIGNAME ) 
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    Complement = Dict( );
    Complement[ "NVoltValues" ] = RowCount;
    Complement[ "stdΔV" ] = σΔV;
    Complement[ "Discarded" ] = Discarded;
    RAWNAME = joinpath( PATHVOLTAGE, BINNAME );
    save( RAWNAME, "RAW", Float16.( BINRAW ), "Complement", Complement );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
end

In [ ]:
# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
using JLD
@suppress begin
    PATHFunctions = "/home/isabel/Dropbox/git-repos/All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# FILEBRW = "/run/media/isabel/Data/Medula/28-02-2022/BRWs/Phase_01.brw";
FILEBRW = "/run/media/isabel/Data/Hipocampo/Espontanea/01-06-2022/BRWs/Phase_10.brw";
FILEVARS = FindContent( "Variables.jld", FILEBRW );
Variables = LoadDict( FILEVARS );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

#  ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ Metricas ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
# 1. Number of voltage values present on each channel            # UniqueCount
# 2. Direct standard deviation                                   # std
# 3. Standard deviation of voltage change with Δt variable       # stdΔV
# 4. Global threshold                                            # Donoho * SigmaData
# 5. Standard Error                                              # xerror
# 6. Static Threshold Events                                     # nSTEvents
# 7. Dynamic Threshold Events                                    # nDTEvents
# 8. Medians 
# 9. Means
# 10. Z-score
Metrics = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ];

# 1. Raw data, Frequencies = [0, SamplingRate/2]
# 2. MultiUnit Activity, Frequencies = [ MUAlF, MUAHF ] ( ~ 300, 5000 Hz )
# 2. Local Field Potential, Frequencies = [ LFPlF, LFPHF ] ( ~ 2, 300 Hz )
Filters = [ 1, 2, 3 ];

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~ Parameters ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
Parameters = Dict(
    # for detecting saturations
    "maxvolt"              => 3000, # μV
    "minvolt"              => -3000, # μV
    # for the neighborhood
    "minchannels"          => 8, # needed valid channels on the neighborhood
    "maxradius"            => 4, # maximus radius for the searching neighborhood ( 1 -> 8)
    # how much of the channel can we tolerate being saturated before patching it all
    "limite_saturacion"    => 0.4, # [ 0, 1 ]
    "window"               => 5, # ms, para la deteccion de eventos
    "bit"                  => 1, # ms, deslizamiento de la ventana
    "stokes"               => 0.3, # ms, distancia maxima entre eventos para ser descartados
    "thr"                  => -50, # μV, umbral estatico
    "ΔT"                   => 250, # ms, para calcular el cambio de voltaje
    "MUAlF"                => 290, # Hz, limite inferior de frecuencia para el MUA
    "MUAHF"                => 5010, # Hz, limite superior de frecuencia para el MUA
    "MUAfac"               => 10, # Hz, banda de transicion para el MUA
    "LFPlF"                => 2, # Hz, limite inferior de frecuencia para el LFP
    "LFPHF"                => 302, # Hz, limite superior de frecuencia para el LFP
    "LFPfac"               => 2, # Hz, banda de transicion para el LFP
    "SamplingRate"         => Variables[ "SamplingRate" ]
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

PATHANALITICS = joinpath( dirname( FindContent( "Info", FILEBRW )[ 1 ] ), "Analitics" );
PATHWS = joinpath( PATHANALITICS, "Ws" ); mkpath( PATHWS );
PATHDISCARDED = joinpath( PATHANALITICS, "Discarded" ); mkpath( PATHDISCARDED );
PATHINDEXES = joinpath( PATHANALITICS, "SpikeIndexes" ); mkpath( PATHINDEXES );
PATHSPECTROS = joinpath( PATHANALITICS, "Spectros" ); mkpath( PATHSPECTROS );
PATHCSDAK = joinpath( PATHANALITICS, "CSDAK" ); mkpath( PATHCSDAK );
PATHCSDAI = joinpath( PATHANALITICS, "CSDAI" ); mkpath( PATHCSDAI );

FILESVOLTAGE = FindContent( "Voltage", FILEBRW );
Σ = length( FILESVOLTAGE )

for σ = 1:Σ
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    FILERAW = FILESVOLTAGE[ σ ];
    BINNAME = basename( FILERAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    BINRAW = Float64.( LoadDict( FILERAW ) );
    BINRAW, Discarded = RemoveSaturationExtrema( Parameters, BINRAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    FILEDISCARDED = joinpath( PATHDISCARDED, BINNAME );
    save( FILEDISCARDED, "Data", Discarded );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    Empties = Discarded[ "Empties" ];
    Ms = [ ];
    AllSpikes = [ ];
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    for f in Filters
        # ~ ~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
        if f == 1
            R, SpikeIndexes = AllMs( Metrics, BINRAW, Parameters, Empties );
            push!( Ms, R ); push!( AllSpikes, SpikeIndexes );
        end
        # ~ ~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
        if f == 2
            MUAlF = Parameters[ "MUAlF" ];
            MUAHF = Parameters[ "MUAHF" ];
            MUAfac = Parameters[ "MUAfac" ];
            data = round.( MatrixFilter( 
                    Parameters, BINRAW, MUAlF, MUAHF, MUAfac, "BPF" ), digits = 3 );
            R, SpikeIndexes = AllMs( Metrics, data, Parameters, Empties );
            push!( Ms, R ); push!( AllSpikes, SpikeIndexes );
        end
        # ~ ~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
        if f == 3
            LFPlF = Parameters[ "LFPlF" ];
            LFPHF = Parameters[ "LFPHF" ];
            LFPfac = Parameters[ "LFPfac" ];
            data = round.( MatrixFilter( 
                    Parameters, BINRAW, LFPlF, LFPHF, LFPfac, "BPF" ), digits = 3 );
            R, SpikeIndexes = AllMs( Metrics, data, Parameters, Empties );
            push!( Ms, R ); push!( AllSpikes, SpikeIndexes );
        end
        # ~ ~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
    end
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    FILEWS = joinpath( PATHWS, BINNAME );
    FILEINDEXES = joinpath( PATHINDEXES, BINNAME );
    save( FILEWS, "Data", Ms );
    save( FILEINDEXES, "Data", AllSpikes );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    S = PFT_mt( Parameters, BINRAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    FILESPECTROS = joinpath( PATHSPECTROS, BINNAME );
    save( FILESPECTROS, "Data", S );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    CSDK = CSDAK( BINRAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    FILECSDAK = joinpath( PATHCSDAK, BINNAME );
    save( FILECSDAK, "Data", Float16.( CSDK ) );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    CSDI = CSDAI( BINRAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    FILECSDAI = joinpath( PATHCSDAI, BINNAME );
    save( FILECSDAI, "Data", Float16.( CSDI ) );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    save( FILERAW, "data", Float16.( BINRAW ) )
    println( "File: ", σ, " of ", Σ, " done" )
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
end

In [ ]:
# funcion que detecte elementos continuos, donde continuidad son incrementos de tamaño predeterminado.
entry = rand( (1:1000), 100 );
step = 20;
testing = copy( entry );
entry = sort( entry );
entry
entry ./ 10

In [ ]:
function Paso03( FILEBRW::String, Parameters::Dict, Metrics::Vector, Filters::Vector )
    FILESVOLTAGE = FindContent( "Voltage", FILEBRW );
    FILEVARS = FindContent( "Variables.jld", FILEBRW );
    FILESTEP3Ws = replace( FILEVARS, "Variables" => "Metrics" );
    FILESPIKES = replace( FILESTEP3Ws, "Metrics" => "SpikesIndexes" );
    FILESPECTROS = replace( FILESPIKES, "SpikesIndexes" => "Spectros" );
    FILEPARAMS = replace( FILEVARS, "Variables" => "Parameters" );
    # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
    AllWs = [ ];
    AllDiscarded = [ ];
    AllIndexes = [ ];
    AllSpectros = [ ];
    # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
    Σ = length( FILESVOLTAGE );
    for σ = 1:Σ
        Ms = [ ];
        AllSpikes = [ ];
        FILERAW = FILESVOLTAGE[ σ ];
        BINRAW = Float64.( LoadDict( FILERAW ) );
        BINRAW, Discarded = RemoveSaturationExtrema( Parameters, BINRAW );
        push!( AllDiscarded, Discarded )
        save( FILERAW, "data", Float16.( BINRAW ) )
        Empties = Discarded[ "Empties" ];
        if isempty( Empties )
            Empties = [ 1 ];
        end
        # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~  #

        # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~  #
        for f in Filters
            if f == 1
                R, SpikeIndexes = AllMs( Metrics, BINRAW, Parameters, Empties );
                push!( Ms, R ); push!( AllSpikes, SpikeIndexes );
            end
            if f == 2
                MUAlF = Parameters[ "MUAlF" ];
                MUAHF = Parameters[ "MUAHF" ];
                MUAfac = Parameters[ "MUAfac" ];
                data = round.( MatrixFilter( Parameters, BINRAW, MUAlF, MUAHF, MUAfac, "BPF" ), digits = 3 );
                R, SpikeIndexes = AllMs( Metrics, data, Parameters, Empties );
                push!( Ms, R ); push!( AllSpikes, SpikeIndexes );
            end
            if f == 3
                LFPlF = Parameters[ "LFPlF" ];
                LFPHF = Parameters[ "LFPHF" ];
                LFPfac = Parameters[ "LFPfac" ];
                data = round.( MatrixFilter( Parameters, BINRAW, LFPlF, LFPHF, LFPfac, "BPF" ), digits = 3 );
                R, SpikeIndexes = AllMs( Metrics, data, Parameters, Empties );
                push!( Ms, R ); push!( AllSpikes, SpikeIndexes );
            end
        end
        # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~  #

        # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~  #
        S = PFT_mt( Parameters, BINRAW );
        # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~  #

        # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~  #
        push!( AllWs, Ms );
        push!( AllIndexes, AllSpikes );
        push!( AllSpectros, S );
        # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~  #
        println( "File: ", σ, " done" )
    end
    # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #

    # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
    Ws = Dict(
        "AllWs" => AllWs,
        "AllDiscarded" => AllDiscarded
    );
    # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #

    # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
    save( FILESTEP3Ws, "Ws", Ws )
    save( FILESPIKES, "SpikesIndexes", AllIndexes )
    save( FILESPECTROS, "Spectros", AllSpectros )
    save( FILEPARAMS, "Parameters", Parameters )
    # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
    return Ws, AllIndexes, AllSpectros
    # ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
end

In [ ]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
limupper = 0.6; # GB maximum size of each segment considering Float16 format
ΔT = 250; # ms para el ΔV
sigma = 3; # para la convolucion gaussiana
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Constants = Dict(
    # for detecting saturations
    "maxvolt" => 3000, # μV
    "minvolt" => -3000, # μV
    # for the neighborhood
    "minchannels" => 8, # needed valid channels on the neighborhood
    "maxradius" => 4, # maximus radius for the searching neighborhood ( 1 -> 8)
    # how much of the channel can we tolerate being saturated before patching it all
    "limite_saturacion" => 0.4, 
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
using JLD
using Plots
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@suppress begin
    # PATHFunctions = "/home/isabel/Dropbox/git-repos/All-Process";
    PATHFunctions = raw"C:\Users\Siby\Dropbox\git-repos\All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# FILEBRW = "/run/media/isabel/Data/Medula/28-02-2022/BRWs/Phase_01.brw";
FILEBRW = raw"C:\Users\Siby\Desktop\Datos\Medula\28-02-2022\BRWs\Phase_01.brw";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Variables, FILEPATHS, FILEVARS = GetVarsHDF5( FILEBRW );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
σ = ChunkSizeSpace( Variables, limupper );
n0s = length( string( σ ) );
nChs = Variables[ "nChs" ];
channels = collect( 1:nChs );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS = load( FILEPATHS )[ "PATHS" ];
PATHMAIN = PATHS[ "PATHMAIN" ];
PATHFIGURES = joinpath( PATHMAIN, "Figures" ); mkpath( PATHFIGURES );
PATHFIGS_TEMP = joinpath( PATHFIGURES, "Step0" ); mkpath( PATHFIGS_TEMP );
PATHVOLTAGE = joinpath( PATHMAIN, "Voltage" ); mkpath( PATHVOLTAGE );
PATHINFO =  PATHS[ "PATHINFO" ];
FILESTEP0Ws = joinpath( PATHINFO, "Step0Ws.jld" );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS[ "PATHFIGURES" ] = PATHFIGURES;
PATHS[ "PATHVOLTAGE" ] = PATHVOLTAGE;
PATHS[ "PATHFIGS_TEMP" ] = PATHFIGS_TEMP;
PATHS[ "FILESTEP0Ws" ] = FILESTEP0Ws;
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
save( FILEPATHS, "PATHS", PATHS );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Empties = Array{ Any }( undef, σ );
Frames = Array{ Any }( undef, σ );
Channels = Array{ Any }( undef, σ );
NVRs = Array{ Any }( undef, σ );
NVPs = Array{ Any }( undef, σ );
DSRs = Array{ Any }( undef, σ );
DSPs = Array{ Any }( undef, σ );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
for n = 1:σ;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW = OneSegment( Variables, n, σ ); # bin in uInt format
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW = Digital2Analogue( Variables, BINRAW ); # bin in μV
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINNAME = string( "BIN", lpad( n, n0s, "0" ), ".jld" );
    FILEBIN = joinpath( PATHVOLTAGE, BINNAME );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    # Pre eliminacion de los extremos
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    σΔV = stdΔV( Variables, BINRAW, ΔT );
    RowCount = UniqueCount( BINRAW );
    data = RemoveInfs( log.( σΔV ) ); 
    P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
    h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
    data = RemoveInfs( log.( RowCount ) ); 
    P3 = Zplot(  data, "W", :bluesreds ); P3 = title!( "# V values" )
    h = copy( RowCount ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
    PF1 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
    Complement = Dict( );
    Complement[ "NVoltValuesCS" ] = RowCount;
    Complement[ "stdΔVCS" ] = σΔV;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW, Discarded = RemoveSaturationExtrema( Variables, Constants, BINRAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    # Post eliminacion de los extremos
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    σΔV = stdΔV( Variables, BINRAW, ΔT );
    RowCount = UniqueCount( BINRAW );
    data = RemoveInfs( log.( σΔV ) ); 
    P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
    h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
    data = RowCount; 
    P3 = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); P3 = title!( "# V values" )
    h = copy( RowCount ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
    PF2 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
    Complement[ "NVoltValuesNS" ] = RowCount;
    Complement[ "stdΔVNS" ] = σΔV;
    Complement[ "Discarded" ] = Discarded;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    FFS0 = plot( PF1, PF2, layout = ( 2, 1 ) );
    FIGNAME = joinpath( PATHFIGS_TEMP, BINNAME );
    FIGNAME = replace( FIGNAME, ".jld" => ".svg" );
    savefig( FFS0, FIGNAME ) 
    RAWNAME = joinpath( PATHVOLTAGE, BINNAME );
    save( RAWNAME, "RAW", Float16.( BINRAW ) );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    Empties[ n ] = vcat( Complement[ "Discarded" ][ "Empties" ]... );
    Frames[ n ] = vcat( Complement[ "Discarded" ][ "Frames" ]... );
    Channels[ n ] = vcat( Complement[ "Discarded" ][ "Channels" ]... );
    NVRs[ n ] = Complement[ "NVoltValuesCS" ];
    NVPs[ n ] = Complement[ "NVoltValuesNS" ];
    DSRs[ n ] = Complement[ "stdΔVNS" ];
    DSPs[ n ] = Complement[ "stdΔVCS" ];
    println( n )
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Step0Ws = Dict(
    "Empties"  => Empties,
    "Frames"   => Frames,
    "Channels" => Channels,
    "NVRs"     => NVRs,
    "NVPs"     => NVPs,
    "DSRs"     => DSRs,
    "DSPs"     => DSPs
);
save( FILESTEP0Ws, "Step0Ws", Step0Ws );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

In [ ]:
# Pasos de Preprocesamiento 

# Checando 

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
lF, HF, fac = ( 290, 5010, 10 ); # MUA
data = round.(
    MatrixFilter( Variables, BINPATCH, lF, HF, fac, "BPF" ), digits = 3 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
for j in 1:nChs
    thr = Donoho( data[ j, : ] ) * SigmaData( data[ j, : ] );
    ch = data[ j, : ];
    st = findall( ch .> thr );
    push!( ST, st );
    push!( NST, length( st ) );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
aux = vcat( ST... );
K = collect( keys( countmap( aux ) ) );
V = collect( values( countmap( aux ) ) );
PosibleDischarges = K[ V .>= Donoho( V ) * SigmaData( V ) ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
for j = 1:nChs
    aux = findall( ST[ j ] .∈ [ PosibleDischarges ] );
    if !isempty( aux )
        push!( PosibleDischargesChannels, j )
        push!( PosibleDischargesChannelsPositive, aux )
    end
end
Z = zeros( Int,nChs );
Z[ PosibleDischargesChannels ] = length.( PosibleDischargesChannelsPositive );
aux = RemoveInfs( log.( Z ) );
F2 = Zplot( aux, "W", :bluesreds );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

F3 = plot( F1, F2, aspect_ratio = 1, wsize = ( 800, 400 ) );

FigCS = NVRs[ segments[ I ] ];
FigSS = UniqueCount( BINRAW );
FigSA = UniqueCount( BINPATCH );

sigma = 3; # para la convolucion gaussiana

P = Array{ Plots.Plot{ Plots.GRBackend } }( undef, 6 );
data = FigCS; 
P[ 1 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 1 ] = title!( "Original" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 2 ] = Zplot( data, "W", :bluesreds ); 
P[ 2 ] = title!( "Original con convolucion" )

data = FigSS; 
P[ 3 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 3 ] = title!( "Sin Saturacion" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 4 ] = Zplot( data, "W", :bluesreds ); 
P[ 4 ] = title!( "Sin Saturacion con convolucion " )

data = FigSA; 
P[ 5 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 5 ] = title!( "Sin Artefactos" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 6 ] = Zplot( data, "W", :bluesreds ); 
P[ 6 ] = title!( "Sin Artefactos con convolucion" );

F4 = plot( P..., layout = ( 3, 2 ), wsize = ( 800, 600 ), cbar = :none, titlefontsize = 10 )

Step0Ws = load( FILESTEP0Ws )[ "Step0Ws" ];
NVRs = Step0Ws[ "NVRs" ];
#NVRs[ 10 ] = FigSA;
aux01 = std.( NVRs );
thr = mean( aux01 ) + 2*std( aux01 );
segments = findall( aux01 .>= thr )









# Hacer Funcion
AllFrames = AllFrames[ sortperm( AllChannels ) ];
AllChannels = sort( AllChannels );
ValidChs = setdiff( channels, AllChannels );
for emptie in Empties
    radius = 1;
    _, neighs = Neighbors( emptie, radius );
    ValidNeighbors = [ ];
    ValidNeighbors = intersect( neighs, ValidChs );
    A = ( length( ValidNeighbors ) .>= minchannels );
    while !A && radius <= maxradius
        radius = radius + 1;
        _, neighs = Neighbors( emptie, radius );
        ValidNeighbors = intersect( neighs, ValidChs );
        A = ( length( ValidNeighbors ) .>= minchannels );
    end
    BIN[ emptie, : ] = mean( BIN[ ValidNeighbors, : ], dims = 1 );
end

In [ ]:
# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ funciones ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Metrics = [ 1, 3, 10 ];
# Metrics = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ];
# 1. Raw Data
# 2. MUA
# 3. LFP
Signals = [ 1, 2, 3 ];
# ~~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# para la reparacion de las saturaciones
Constants = Dict(
    # for detecting saturations
    "maxvolt"              => 3000, # μV
    "minvolt"              => -3000, # μV
    # for the neighborhood
    "minchannels"          => 8, # needed valid channels on the neighborhood
    "maxradius"            => 4, # maximus radius for the searching neighborhood ( 1 -> 8)
    # how much of the channel can we tolerate being saturated before patching it all
    "limite_saturacion"    => 0.4 # 0 -> 1
);
# para la deteccion de espigas
parameters = Dict(
    "window"               => 5, # ms, para la deteccion de eventos
    "bit"                  => 1, # ms, deslizamiento de la ventana
    "distance"             => 0.3, # ms, distancia maxima entre eventos para ser descartados
    "thr"                  => -50, # μV, umbral estatico
    #
    "ΔT"                   => 250, # ms, para calcular el cambio de voltaje
    #
    "MUAlF"                => 290, # Hz, limite inferior de frecuencia para el MUA
    "MUAHF"                => 5010, # Hz,
    "MUAfac"               => 10, # Hz,
    #
    "LFPlF"                => 2, # Hz,
    "LFPHF"                => 302, # Hz,
    "LFPfac"               => 2 # Hz,
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
nChs = Variables[ "nChs" ];
channels = collect( 1:nChs );
Σ = length( FILESVOLTAGE );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# nWs = length( Metrics );
# nSs = length( Signals );
Wn = Array{ Float64 }( undef, nChs,  Σ ); fill!( Wn, 0 );
sn = 1;
σ = 1;
FILERAW = FILESVOLTAGE[ σ ];
BINRAW = Float64.( LoadDict( FILERAW ) );
@time BINRAW, Discarded = RemoveSaturationExtrema( Variables, Constants, BINRAW );
@time BINRAW = round.( BINRAW, digits = 3 );
wn = 1;
data = BINRAW;
nwn = 1;
Wn[ :, nwn ] = UniqueCount( data );
using StatsBase
using BinningAnalysis
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
AllWs = Array{ Matrix{Float64} }( undef,  Σ );
AllEventsRAWStat = Array{ Vector{Any} }( undef,  Σ );
AllEventsMUAStat = Array{ Vector{Any} }( undef,  Σ );
AllEventsLFPStat = Array{ Vector{Any} }( undef,  Σ );
AllEventsRAWDin = Array{ Vector{Any} }( undef,  Σ );
AllEventsLFPDin = Array{ Vector{Any} }( undef,  Σ );
AllEventsMUADin = Array{ Vector{Any} }( undef,  Σ );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
σ = 1;
Ws = zeros( Float64, nChs, 3*nWs );
FILERAW = FILESVOLTAGE[ σ ];
BINRAW = load( FILERAW );
BINRAW = Float64.( BINRAW[ collect( keys( BINRAW ) )[ 1 ] ] );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@time BINRAW, Discarded = RemoveSaturationExtrema( Variables, Constants, BINRAW );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@time BINMUA = round.(
        MatrixFilter( Variables, BINRAW, MUAlF, MUAHF, MUAfac, "BPF" ), digits = 3 );
@time BINLFP = round.(
        MatrixFilter( Variables, BINRAW, LFPlF, LFPHF, LFPfac, "BPF" ), digits = 3 );
@time BINRAW = round.( BINRAW, digits = 3 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# eventos por canal
# depuracion por ventana
# eventos totales por matriz
Reparaciones = load( FILEREPARACIONES )[ "Data" ];
FILEBRW = "/run/media/isabel/Data/Medula/28-02-2022/BRWs/Phase_01.brw";
FILEREPARACIONES = FindContent( "Reparaciones", FILEBRW )[ 1 ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
"""
    EventsXBin( bin::Vector{Float64}, thr::Real, parameters::Dict{String, Int64} )
        -> Index::Vector{Int64}
"""
function EventsXBin( bin::Vector{Float64}, thr::Real, Variables, parameters::Dict{String, Int64} )
    distance = parameters[ "distance" ];
    distance = ms2Frs( Variables, distance );
    ST = findall( bin .<= thr ); # events that pass the threshold
    index_parcial = [ ];
    if !isempty( ST )
        a = 1;
        while a == 1
            distances = diff( ST ); # distance between them
            nears = findall( distances .<= distance ) .+ 1; # which are close
            if isempty( nears )
                a = 0;
            else
                remove = zeros( Int, size( nears, 1 ) );
                for i = 1:size( nears, 1 )
                    # if the first is less than the second, remove the second
                    if isless( ( bin[ ST[ nears[ i ] ] ] ),
                            ( bin[ ST[ nears[ i ] - 1 ] ] ) )
                        remove[ i ] = nears[ i ] - 1;
                    else
                        remove[ i ] = nears[ i ];
                    end
                end
                if size( remove, 1 ) > 1
                    ST[ unique( remove ) ] .= 0;
                    filter!( x -> x != 0, ST );
                else
                    ST = ST[ Bool.( ST .!= ST[ remove[ 1 ] ] ) ];
                end
            end
        end
        push!( index_parcial, ST )
    else
        push!( index_parcial, [ ] )
    end
    return sort( unique( vcat( index_parcial... ) ) )
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
"""
    EventsXThrs( channel::Vector{Float64}, thr::Real, parameters::Dict{String, Int64} )
        -> index::Vector{Int64}
        Index of detected suprathreshold events with a pre-established threshold
        using EventsXBin
"""
function EventsXThrs( channel::Vector{Float64}, thr::Real, parameters::Dict{String, Int64} )
    index = EventsXBin( channel, thr, Variables, parameters );
    return index
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
"""
    EventsXChannel( channel::Vector{Float64}, parameters::Dict{String, Int64} )
        -> thrs::Vector{Float64}, index_real::Vector{Inf64}
        using Donoho, EventsXBin, ms2Frs
"""
function EventsXChannel( channel::Vector{Float64}, Variables, parameters::Dict{String, Int64} )
    window = parameters[ "window" ];
    bit = parameters[ "bit" ];
    window = ms2Frs( Variables, window );
    bit = ms2Frs( Variables, bit ),
     Σ = sqrt( 2*log( length( channel ) ) );
    index_real = [ ];
    i = 1;
    I = Int( ( ( ( i - 1 ) * bit ) + 1 ) ); J = Int( I + window - 1 );
    thrs = [ ];
    thr = 0;
    while J <= ( length( channel ) - Int( window - 1 ) )
        bin = channel[ I:J ];
        if !iszero( bin )
            thr = -1* Σ*abs( Donoho( bin ) );
            index_parcial = EventsXBin( bin, thr, parameters );
            if !isempty( index_parcial )
                index_real = vcat( index_real, ( index_parcial .+ I .- 1 ) );
            end
        end
        i = i + 1;
        I = Int( ( ( ( i - 1 ) * bit ) + 1 ) ); J = Int( I + window - 1 );
    end
    index_real = unique( index_real );
    return index_real
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
function AllWs( BINRAW )

Ws[ :, 1 ] = UniqueCount( BINRAW );
Ws[ :, 2 ] = UniqueCount( BINLFP );
Ws[ :, 3 ] = UniqueCount( BINMUA );

Ws[ :, 4 ] = std( BINRAW, dims = 2 );
Ws[ :, 5 ] = std( BINLFP, dims = 2 );
Ws[ :, 6 ] = std( BINMUA, dims = 2 );

Ws[ :, 7 ] = stdΔV( Variables, BINRAW, ΔT );
Ws[ :, 8 ] = stdΔV( Variables, BINLFP, ΔT );
Ws[ :, 9 ] = stdΔV( Variables, BINMUA, ΔT );

constant = SigmaData( BINRAW[ 1, : ] );
[ Ws[ k , 10 ] = Donoho( BINRAW[ k, : ] ) * constant for k in channels ];
[ Ws[ k , 11 ] = Donoho( BINLFP[ k, : ] ) * constant for k in channels ];
[ Ws[ k , 12 ] = Donoho( BINMUA[ k, : ] ) * constant for k in channels ];

for k in channels
    _, Ws[ k, 13 ] = jackknife( identity, BINRAW[ k, : ] );
    _, Ws[ k, 14 ] = jackknife( identity, BINLFP[ k, : ] );
    _, Ws[ k, 15 ] = jackknife( identity, BINMUA[ k, : ] );
end

EventsRAWStat = [ ]; EventsMUAStat = [ ]; EventsLFPStat = [ ];
for i in channels
    push!( EventsRAWStat, EventsXThrs( BINRAW[ i, : ], thr, parameters ) );
    push!( EventsLFPStat, EventsXThrs( BINLFP[ i, : ], thr, parameters ) );
    push!( EventsMUAStat, EventsXThrs( BINMUA[ i, : ], thr, parameters ) );
end
AllEventsRAWStat[ σ ] = EventsRAWStat;
AllEventsLFPStat[ σ ] = EventsLFPStat;
AllEventsMUAStat[ σ ] = EventsMUAStat;

EventsRAWDin = [ ]; EventsLFPDin = [ ]; EventsMUADin = [ ]; 
for i in channels
    push!( EventsRAWDin, EventsXChannel( BINRAW[ i, : ], parameters ) );
    push!( EventsLFPDin, EventsXChannel( BINMUA[ i, : ], parameters ) );
    push!( EventsMUADin, EventsXChannel( BINLFP[ i, : ], parameters ) );
end
AllEventsRAWDin[ σ ] = EventsRAWDin;
AllEventsLFPDin[ σ ] = EventsLFPDin;
AllEventsMUADin[ σ ] = EventsMUADin;

Ws[ :, 16 ] = length.( EventsRAWStat );
Ws[ :, 17 ] = length.( EventsMUAStat );
Ws[ :, 18 ] = length.( EventsLFPStat );

Ws[ :, 19 ] = length.( EventsRAWDin );
Ws[ :, 20 ] = length.( EventsMUADin );
Ws[ :, 21 ] = length.( EventsLFPDin );

Ws[ :, 22 ] = median( BINRAW, dims = 2 );
Ws[ :, 23 ] = median( BINMUA, dims = 2 );
Ws[ :, 24 ] = median( BINLFP, dims = 2 );

Ws[ :, 25 ] = mean( BINRAW, dims = 2 );
Ws[ :, 26 ] = mean( BINMUA, dims = 2 );
Ws[ :, 27 ] = mean( BINLFP, dims = 2 );

[ Ws[ k , 28 ] = sum( abs.( zscore( BINRAW[ k, : ] ) ) ) for k in channels ];
[ Ws[ k , 29 ] = sum( abs.( zscore( BINLFP[ k, : ] ) ) ) for k in channels ];
[ Ws[ k , 30 ] = sum( abs.( zscore( BINMUA[ k, : ] ) ) ) for k in channels ];

GeneralPlots = Array{ Plots.Plot{ Plots.GRBackend } }( undef, nWs );
W0NTotal = Array{ Float64 }( undef, nChs, nWs );

FILEWs = joinpath( PATHS[ "PATHINFO" ], "AllWs.jld" );
save( FILEWs, "Wn", Wn, "W0NTotal", W0NTotal );

# UniqueCount
# std
# stdΔV
# Donoho * SigmaData
# xerror
# nSTEvents
# nDTEvents
# Medians
# Means
# zscore

In [ ]:
# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
@suppress begin
    using JLD
    using StatsBase
    using Plots
    # ~~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

    # ~~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    PATHFunctions = raw"C:\Users\Siby\Dropbox\git-repos\All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILEBRW = raw"C:\Users\Siby\Desktop\Datos\Medula\28-02-2022\BRWs\Phase_01.brw";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILESVOLTAGE = FindContent( "Voltage", FILEBRW );
PATHINFO =  dirname( FindContent( "Info", FILEBRW )[ 1 ] );
FILESTEP0Ws = joinpath( PATHINFO, "Step0Ws.jld" );
FILESRAW = FindContent( "RAW", FILEBRW );
FILEVARIABLES = FindContent( "Variables.jld", FILEBRW )[ 1 ];
Variables = load( FILEVARIABLES )[ "Variables" ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
nChs = Variables[ "nChs" ];
thrs = [ ]; ST = [ ];
PosibleDischargesChannels = [ ]; 
PosibleDischargesChannelsPositive = [ ];
maximum_bin = 6; # msec
maximum_bin = ms2Frs( Variables, maximum_bin );
SamplingRate = Variables[ "SamplingRate" ];
lF, HF, fac = ( 290, 5010, 10 ); # MUA
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Step0Ws = load( FILESTEP0Ws )[ "Step0Ws" ];
Empties = Step0Ws[ "Empties" ]; 
Frames = Step0Ws[ "Frames" ]; 
Channels = Step0Ws[ "Channels" ];
NVRs = Step0Ws[ "NVRs" ]; 
AllDiscardedChannels = sort( unique( vcat( Empties... ) ) ); 
aux01 = std.( NVRs );
thr = mean( aux01 ) + 2*std( aux01 );
segments = findall( aux01 .>= thr );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #



# Comprobando

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
thrs = [ ]; 
ST = [ ]; 
PosibleDischargesChannels = [ ]; 
PosibleDischargesChannelsPositive = [ ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
lF, HF, fac = ( 290, 5010, 10 ); # MUA
data = round.(
    MatrixFilter( Variables, BINPATCH, lF, HF, fac, "BPF" ), digits = 3 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
for j in 1:nChs
    thr = Donoho( data[ j, : ] ) * SigmaData( data[ j, : ] );
    ch = data[ j, : ];
    st = findall( ch .> thr );
    push!( ST, st );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
aux = vcat( ST... );
K = collect( keys( countmap( aux ) ) );
V = collect( values( countmap( aux ) ) );
PosibleDischarges = K[ V .>= Donoho( V ) * SigmaData( V ) ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
for j = 1:nChs
    aux = findall( ST[ j ] .∈ [ PosibleDischarges ] );
    if !isempty( aux )
        push!( PosibleDischargesChannels, j )
        push!( PosibleDischargesChannelsPositive, aux )
    end
end
Z = zeros( Int, nChs );
Z[ PosibleDischargesChannels ] = length.( PosibleDischargesChannelsPositive );
aux = RemoveInfs( log.( Z ) );
F2 = Zplot( aux, "W", :bluesreds );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
F1_2 = plot( F1, F2, layout = ( 1, 2 ), wsize = ( 800, 400 ) )
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #











# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
FigCS = NVRs[ segments[ I ] ];
FigSS = UniqueCount( BINRAW );
FigSA = UniqueCount( BINPATCH );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
sigma = 3; # para la convolucion gaussiana
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
P = Array{ Plots.Plot{ Plots.GRBackend } }( undef, 6 );
data = FigCS; 
P[ 1 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 1 ] = title!( "Original" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 2 ] = Zplot( data, "W", :bluesreds ); 
P[ 2 ] = title!( "Original con convolucion" )
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
data = FigSS; 
P[ 3 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 3 ] = title!( "Sin Saturacion" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 4 ] = Zplot( data, "W", :bluesreds ); 
P[ 4 ] = title!( "Sin Saturacion con convolucion " )
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
data = FigSA; 
P[ 5 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 5 ] = title!( "Sin Artefactos" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 6 ] = Zplot( data, "W", :bluesreds ); 
P[ 6 ] = title!( "Sin Artefactos con convolucion" );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
F4 = plot( P..., layout = ( 3, 2 ), wsize = ( 800, 600 ), cbar = :none, titlefontsize = 10 )
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #



In [ ]:
# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
@suppress begin
    using JLD
    using StatsBase
    using Plots
    # ~~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

    # ~~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
    PATHFunctions = "/home/isabel/Dropbox/git-repos/All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILEBRW = "/run/media/isabel/Data/Medula/28-02-2022/BRWs/Phase_01.brw";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILESVOLTAGE = FindContent( "Voltage", FILEBRW );
PATHINFO =  dirname( FindContent( "Info", FILEBRW )[ 1 ] );
FILESTEP0Ws = joinpath( PATHINFO, "Step0Ws.jld" );
FILEVARIABLES = FindContent( "Variables.jld", FILEBRW )[ 1 ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
Variables = load( FILEVARIABLES )[ "Variables" ];
nFrames = Int( Variables[ "NRecFrames" ] / length( FILESVOLTAGE ) );
nChs = Variables[ "nChs" ];
maximum_bin = 6; # msec
maximum_bin = ms2Frs( Variables, maximum_bin );
SamplingRate = Variables[ "SamplingRate" ];
lF, HF, fac = ( 290, 5010, 10 ); # MUA
sigma = 3; # para la convolucion gaussiana
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Constants = Dict(
    # for detecting saturations
    "maxvolt" => 3000, # μV
    "minvolt" => -3000, # μV
    # for the neighborhood
    "minchannels" => 8, # needed valid channels on the neighborhood
    "maxradius" => 4, # maximus radius for the searching neighborhood ( 1 -> 8)
    # how much of the channel can we tolerate being saturated before patching it all
    "limite_saturacion" => 0.4, 
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Step0Ws = load( FILESTEP0Ws )[ "Step0Ws" ];
Empties = Step0Ws[ "Empties" ]; 
Frames = Step0Ws[ "Frames" ]; 
Channels = Step0Ws[ "Channels" ];
NVRs = Step0Ws[ "NVRs" ]; 
AllDiscardedChannels = sort( unique( vcat( Empties... ) ) ); 
aux01 = std.( NVRs );
thr = mean( aux01 ) + 2*std( aux01 );
segments = findall( aux01 .>= thr );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
BINPATCH = Array{ Float64 }( undef, nChs, nFrames );
BINRAW = Array{ Float64 }( undef, nChs, nFrames );
# F1 = Plots.Plot{ Plots.GRBackend };
# P1 = Plots.Plot{ Plots.GRBackend };
# P2 = Plots.Plot{ Plots.GRBackend };
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
I = 1;

FILERAW = FILESVOLTAGE[ segments[ I ] ];
BINRAW = Float64.( load( FILERAW )[ "BINRAW" ] );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
data = round.(
    MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
ST = [ ];
aux = [ ];
thrs = [ ];
auxRAW = [ ];
PosibleDischargesChannels = [ ];
PosibleDischargesChannelsPositive = [ ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
for j in 1:nChs
    thr = Donoho( data[ j, : ] ) * SigmaData( data[ j, : ] );
    ch = data[ j, : ];
    st = findall( ch .> thr );
    push!( ST, st );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
auxRAW = vcat( ST... );
K = collect( keys( countmap( auxRAW ) ) );
V = collect( values( countmap( auxRAW ) ) );
PosibleDischarges = K[ V .>= Donoho( V ) * SigmaData( V ) ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
for j = 1:nChs
    auxRAW = findall( ST[ j ] .∈ [ PosibleDischarges ] );
    if !isempty( auxRAW )
        push!( PosibleDischargesChannels, j )
        push!( PosibleDischargesChannelsPositive, auxRAW )
    end
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
Z = zeros( Int, nChs );
Z[ PosibleDischargesChannels ] = length.( PosibleDischargesChannelsPositive );
aux = RemoveInfs( log.( Z ) );
# P1 = Zplot( aux, "W", :bluesreds );
cl0, clN = extrema( aux );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
a = unique( FindAllThrs( aux ) );
a = a[ a .!= 0 ];
TF = minimum( a );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
# SatChannels = findall( aux .>= TF );
# FinalSatChannels = SatChannels[ SatChannels .∉ [ Empties[ segments[ I ] ] ] ];
FinalSatChannels = findall( aux .>= TF );
PosibleDischarges = sort( PosibleDischarges );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
v = diff( PosibleDischarges );
SePasa = OutlayerME( v );
# generando los bines
nSegs = length( SePasa ) + 1;
Indexes = zeros( nSegs, 2 );
SePasa = vcat( 0, SePasa, length( PosibleDischarges ) );
for nS in 1:nSegs
    Indexes[ nS, 1 ] = PosibleDischarges[ SePasa[ nS ] + 1 ]
    Indexes[ nS, 2 ] = PosibleDischarges[ SePasa[ nS + 1 ] ]
end
var0 = sort( vcat( Indexes... ) );
var1 = diff( var0 );
none = collect( 1:length( var1 ) ) .% 2;
par = none .!= 1;
length_bin = var1[ Bool.( none ) ];
distance_bin = var1[ Bool.( par ) ];
Ol = OutlayerME( length_bin );
quitar = [ ];
if !isempty( Ol )
    for ol in Ol
        push!( quitar, ol*2 - 1 );
        push!( quitar, ol*2 );
    end
end
quitar = sort( unique( vcat( quitar... ) ) );
var2 = Indexes[ Indexes .∉ [ var0[ quitar ] ] ];
FinalBines = Int.( reshape( var2, Int( length( var2 ) / 2 ), 2 ) );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
BINPATCH = Float64.( load( FILERAW )[ "BINRAW" ] );
AllAllBines = [ ];
for channel in FinalSatChannels
    AllBines = [ ]; 
    for fb = 1:size( FinalBines, 1 )
        Energias = [ ];
        x00 = FinalBines[ fb, 1 ];
        xF0 = FinalBines[ fb, 2 ];
        lb = xF0 - x00;
        falta = maximum_bin - lb;
        A = ( falta != 0 );
        B = ( ( xF0 + falta ) <= size( data, 2 ) );
        C = ( ( x00 - falta ) >= 1 );
        if A && B && C
            counter = 0;
            while counter <= falta
                x0n = x00 - counter;
                xFn = xF0 + falta - counter;
                signal = data[ channel, x0n:xFn ];
                push!( Energias, [ Energy( signal, SamplingRate ), x0n, xFn ] )
                counter = counter + 1;
            end
            else
                println( "no se puede mover la ventana" );
                println( "bin:", fb, "canal: ", channel );
            break
        end
        Energias = Array( hcat( Energias... )' );
        results = Energias[ Energias[ :, 1 ] .== maximum( Energias[ :, 1 ] ), : ];
        push!( AllBines, results );
    end
    AllBines = vcat( AllBines... );
    temp = Int.( vcat( AllBines[ :, 2 ],  AllBines[ :, 3 ] ) );
    AllFrames = [ ];
    for fr = 1:size( AllBines, 1 )
        push!( AllFrames, collect( AllBines[ fr, 2 ]:AllBines[ fr, 3 ] ) );
    end
    AllFrames = sort( Int.( vcat( AllFrames... ) ) );
    ValidFrames = setdiff( collect( 1:size( data, 2 ) ), AllFrames );
    nsamples = length( AllFrames );
    BINPATCH[ channel, AllFrames ] = BINPATCH[ channel, sample( ValidFrames, nsamples ) ];
    push!( AllAllBines, AllBines );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
println( "Segment ", segments[ I ], " ready" );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# Comprobando
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
ST = [ ];
aux = [ ];
thrs = [ ];
auxPATCH = [ ];
PosibleDischargesChannels = [ ];
PosibleDischargesChannelsPositive = [ ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
data = [ ];
data = round.(
    MatrixFilter( Variables, BINPATCH, lF, HF, fac, "BPF" ), digits = 3 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
ST = [ ];
thrs = [ ]; 
PosibleDischargesChannels = [ ]; 
PosibleDischargesChannelsPositive = [ ];
for j in 1:nChs
    thr = Donoho( data[ j, : ] ) * SigmaData( data[ j, : ] );
    ch = data[ j, : ];
    st = findall( ch .> thr );
    push!( ST, st );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
auxPATCH = vcat( ST... );
K = collect( keys( countmap( auxPATCH ) ) );
V = collect( values( countmap( auxPATCH ) ) );
PosibleDischarges = K[ V .>= Donoho( V ) * SigmaData( V ) ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
for j = 1:nChs
    auxPATCH = findall( ST[ j ] .∈ [ PosibleDischarges ] );
    if !isempty( auxPATCH )
        push!( PosibleDischargesChannels, j )
        push!( PosibleDischargesChannelsPositive, auxPATCH )
    end
end
Z = zeros( Int, nChs );
Z[ PosibleDischargesChannels ] = length.( PosibleDischargesChannelsPositive );
aux = RemoveInfs( log.( Z ) );
#P2 = Zplot( aux, "W", :bluesreds );
cl00, clNN = extrema( aux );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
clims = ( minimum( [ cl0, cl00 ] ), maximum( [ clN, clNN ] ) );
# F1 = plot( P1, P2, layout = ( 1, 2 ), wsize = ( 800, 400 ), clims = clims );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
FigCS = NVRs[ segments[ I ] ];
FigSS = UniqueCount( BINRAW );
FigSA = UniqueCount( BINPATCH );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
P = Array{ Plots.Plot{ Plots.GRBackend } }( undef, 6 );
data = FigCS; 
P[ 1 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 1 ] = title!( "Original" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 2 ] = Zplot( data, "W", :bluesreds ); 
P[ 2 ] = title!( "Original con convolucion" )
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
data = FigSS; 
P[ 3 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 3 ] = title!( "Sin Saturacion" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 4 ] = Zplot( data, "W", :bluesreds ); 
P[ 4 ] = title!( "Sin Saturacion con convolucion " )
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
data = FigSA; 
P[ 5 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 5 ] = title!( "Sin Artefactos" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 6 ] = Zplot( data, "W", :bluesreds ); 
P[ 6 ] = title!( "Sin Artefactos con convolucion" );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
F2 = plot( P..., layout = ( 3, 2 ), wsize = ( 800, 600 ), cbar = :none, titlefontsize = 10 )
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
BINRAW = deepcopy( BINPATCH );
F1 = Plots.Plot{ Plots.GRBackend };
P1 = Plots.Plot{ Plots.GRBackend };
P2 = Plots.Plot{ Plots.GRBackend };
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
I = 1;
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
data = round.(
    MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
ST = [ ];
aux = [ ];
thrs = [ ];
auxRAW = [ ];
PosibleDischargesChannels = [ ];
PosibleDischargesChannelsPositive = [ ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
for j in 1:nChs
    thr = Donoho( data[ j, : ] ) * SigmaData( data[ j, : ] );
    ch = data[ j, : ];
    st = findall( ch .> thr );
    push!( ST, st );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
auxRAW = vcat( ST... );
K = collect( keys( countmap( auxRAW ) ) );
V = collect( values( countmap( auxRAW ) ) );
PosibleDischarges = K[ V .>= Donoho( V ) * SigmaData( V ) ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
for j = 1:nChs
    auxRAW = findall( ST[ j ] .∈ [ PosibleDischarges ] );
    if !isempty( auxRAW )
        push!( PosibleDischargesChannels, j )
        push!( PosibleDischargesChannelsPositive, auxRAW )
    end
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
Z = zeros( Int, nChs );
Z[ PosibleDischargesChannels ] = length.( PosibleDischargesChannelsPositive );
aux = RemoveInfs( log.( Z ) );
P1 = Zplot( aux, "W", :bluesreds );
cl0, clN = extrema( aux );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
a = unique( FindAllThrs( aux ) );
a = a[ a .!= 0 ];
TF = minimum( a );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
# SatChannels = findall( aux .>= TF );
# FinalSatChannels = SatChannels[ SatChannels .∉ [ Empties[ segments[ I ] ] ] ];
FinalSatChannels = findall( aux .>= TF );
PosibleDischarges = sort( PosibleDischarges );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
v = diff( PosibleDischarges );
SePasa = OutlayerME( v );
# generando los bines
nSegs = length( SePasa ) + 1;
Indexes = zeros( nSegs, 2 );
SePasa = vcat( 0, SePasa, length( PosibleDischarges ) );
for nS in 1:nSegs
    Indexes[ nS, 1 ] = PosibleDischarges[ SePasa[ nS ] + 1 ]
    Indexes[ nS, 2 ] = PosibleDischarges[ SePasa[ nS + 1 ] ]
end
var0 = sort( vcat( Indexes... ) );
var1 = diff( var0 );
none = collect( 1:length( var1 ) ) .% 2;
par = none .!= 1;
length_bin = var1[ Bool.( none ) ];
distance_bin = var1[ Bool.( par ) ];
Ol = OutlayerME( length_bin );
quitar = [ ];
if !isempty( Ol )
    for ol in Ol
        push!( quitar, ol*2 - 1 );
        push!( quitar, ol*2 );
    end
end
quitar = sort( unique( vcat( quitar... ) ) );
var2 = Indexes[ Indexes .∉ [ var0[ quitar ] ] ];
FinalBines = Int.( reshape( var2, Int( length( var2 ) / 2 ), 2 ) );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
# BINPATCH = Float64.( load( FILERAW )[ "BINRAW" ] );
AllAllBines = [ ];
for channel in FinalSatChannels
    AllBines = [ ]; 
    for fb = 1:size( FinalBines, 1 )
        Energias = [ ];
        x00 = FinalBines[ fb, 1 ];
        xF0 = FinalBines[ fb, 2 ];
        lb = xF0 - x00;
        falta = maximum_bin - lb;
        A = ( falta != 0 );
        B = ( ( xF0 + falta ) <= size( data, 2 ) );
        C = ( ( x00 - falta ) >= 1 );
        if A && B && C
            counter = 0;
            while counter <= falta
                x0n = x00 - counter;
                xFn = xF0 + falta - counter;
                signal = data[ channel, x0n:xFn ];
                push!( Energias, [ Energy( signal, SamplingRate ), x0n, xFn ] )
                counter = counter + 1;
            end
            else
                println( "no se puede mover la ventana" );
                println( "bin:", fb, "canal: ", channel );
            break
        end
        Energias = Array( hcat( Energias... )' );
        results = Energias[ Energias[ :, 1 ] .== maximum( Energias[ :, 1 ] ), : ];
        push!( AllBines, results );
    end
    AllBines = vcat( AllBines... );
    temp = Int.( vcat( AllBines[ :, 2 ],  AllBines[ :, 3 ] ) );
    AllFrames = [ ];
    for fr = 1:size( AllBines, 1 )
        push!( AllFrames, collect( AllBines[ fr, 2 ]:AllBines[ fr, 3 ] ) );
    end
    AllFrames = sort( Int.( vcat( AllFrames... ) ) );
    ValidFrames = setdiff( collect( 1:size( data, 2 ) ), AllFrames );
    nsamples = length( AllFrames );
    BINPATCH[ channel, AllFrames ] = BINPATCH[ channel, sample( ValidFrames, nsamples ) ];
    push!( AllAllBines, AllBines );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
println( "Segment ", segments[ I ], " ready" );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# Comprobando
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
ST = [ ];
aux = [ ];
thrs = [ ];
auxPATCH = [ ];
PosibleDischargesChannels = [ ];
PosibleDischargesChannelsPositive = [ ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
data = [ ];
data = round.(
    MatrixFilter( Variables, BINPATCH, lF, HF, fac, "BPF" ), digits = 3 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
ST = [ ];
thrs = [ ]; 
PosibleDischargesChannels = [ ]; 
PosibleDischargesChannelsPositive = [ ];
for j in 1:nChs
    thr = Donoho( data[ j, : ] ) * SigmaData( data[ j, : ] );
    ch = data[ j, : ];
    st = findall( ch .> thr );
    push!( ST, st );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
auxPATCH = vcat( ST... );
K = collect( keys( countmap( auxPATCH ) ) );
V = collect( values( countmap( auxPATCH ) ) );
PosibleDischarges = K[ V .>= Donoho( V ) * SigmaData( V ) ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
for j = 1:nChs
    auxPATCH = findall( ST[ j ] .∈ [ PosibleDischarges ] );
    if !isempty( auxPATCH )
        push!( PosibleDischargesChannels, j )
        push!( PosibleDischargesChannelsPositive, auxPATCH )
    end
end
Z = zeros( Int, nChs );
Z[ PosibleDischargesChannels ] = length.( PosibleDischargesChannelsPositive );
aux = RemoveInfs( log.( Z ) );
P2 = Zplot( aux, "W", :bluesreds );
cl00, clNN = extrema( aux );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
clims = ( minimum( [ cl0, cl00 ] ), maximum( [ clN, clNN ] ) );
F1 = plot( P1, P2, layout = ( 1, 2 ), wsize = ( 800, 400 ), clims = clims );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
FigCS = NVRs[ segments[ I ] ];
FigSS = UniqueCount( BINRAW );
FigSA = UniqueCount( BINPATCH );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

F1

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
P = Array{ Plots.Plot{ Plots.GRBackend } }( undef, 6 );
data = FigCS; 
P[ 1 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 1 ] = title!( "Original" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 2 ] = Zplot( data, "W", :bluesreds ); 
P[ 2 ] = title!( "Original con convolucion" )
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
data = FigSS; 
P[ 3 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 3 ] = title!( "Sin Saturacion" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 4 ] = Zplot( data, "W", :bluesreds ); 
P[ 4 ] = title!( "Sin Saturacion con convolucion " )
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
data = FigSA; 
P[ 5 ] = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); 
P[ 5 ] = title!( "Sin Artefactos" )
h = copy( data ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P[ 6 ] = Zplot( data, "W", :bluesreds ); 
P[ 6 ] = title!( "Sin Artefactos con convolucion" );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
F2 = plot( P..., layout = ( 3, 2 ), wsize = ( 800, 600 ), cbar = :none, titlefontsize = 10 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

F2

original : 36.68549757550326
vuelta 01 : 33.11453805256667
vuelta 02 : 33.13439959015859





NVRs = Step0Ws[ "NVRs" ]; 
NVPs = Step0Ws[ "NVPs" ];
aux01 = std.( NVPs );
P1 = bar( aux01, leg = :none );

NVPs[ segments[ I ] ] = FigSA;
aux01 = std.( NVPs );
P2 = bar( aux01, leg = :none );
P3 = plot( P1, P2, layout = ( 2, 1 ) )

NVRs = Step0Ws[ "NVRs" ];
NVPs = Step0Ws[ "NVPs" ];
DSRs = Step0Ws[ "DSRs" ];
DSPs = Step0Ws[ "DSPs" ];
P1 = bar( RemoveInfs( std.( NVRs ) ), leg = :none );
P2 = bar( RemoveInfs( std.( NVPs ) ), leg = :none );
P3 = bar( RemoveInfs( std.( DSRs ) ), leg = :none );
P4 = bar( RemoveInfs( std.( DSPs ) ), leg = :none );
plot( P1, P2, P3, P4, layout = ( 4, 1 ), wsize = ( 800, 800 ), cbar = :none )

# Reparando los empties

Constants = Dict(
    # for detecting saturations
    "maxvolt" => 3000, # μV
    "minvolt" => -3000, # μV
    # for the neighborhood
    "minchannels" => 8, # needed valid channels on the neighborhood
    "maxradius" => 4, # maximus radius for the searching neighborhood ( 1 -> 8)
    # how much of the channel can we tolerate being saturated before patching it all
    "limite_saturacion" => 0.4, 
);

minchannels = Constants[ "minchannels" ];
maxradius = Constants[ "maxradius" ];
nChs = Variables[ "nChs" ];
channels = collect( 1:nChs );
AllChannels = Empties[ segments[ I ] ];
ValidChs = setdiff( channels, AllChannels );

for emptie in AllChannels
    radius = 1;
    _, neighs = Neighbors( emptie, radius );
    ValidNeighbors = [ ];
    ValidNeighbors = intersect( neighs, ValidChs );
    A = ( length( ValidNeighbors ) .>= minchannels );
    while !A && radius <= maxradius
        radius = radius + 1;
        _, neighs = Neighbors( emptie, radius );
        ValidNeighbors = intersect( neighs, ValidChs );
        A = ( length( ValidNeighbors ) .>= minchannels );
    end
    BINPATCH[ emptie, : ] = mean( BINPATCH[ ValidNeighbors, : ], dims = 1 );
    println( emptie, ValidNeighbors )
end

AllChannels

plot(plot(BINPATCH[1,:]),
plot(BINRAW[1,:]), layout = (2, 1))

plot(plot(BINPATCH[1,:]),
plot(BINPATCH[2,:]),
plot(BINPATCH[65,:]),
plot(BINPATCH[66,:]), layout = (4, 1))

# Comprobando
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    data = round.(
        MatrixFilter( Variables, BINPATCH, lF, HF, fac, "BPF" ), digits = 3 );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    for j in 1:nChs
        thr = Donoho( data[ j, : ] ) * SigmaData( data[ j, : ] );
        ch = data[ j, : ];
        st = findall( ch .> thr );
        push!( ST, st );
    end
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    aux = vcat( ST... );
    K = collect( keys( countmap( aux ) ) );
    V = collect( values( countmap( aux ) ) );
    PosibleDischarges = K[ V .>= Donoho( V ) * SigmaData( V ) ];
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    for j = 1:nChs
        aux = findall( ST[ j ] .∈ [ PosibleDischarges ] );
        if !isempty( aux )
            push!( PosibleDischargesChannels, j )
            push!( PosibleDischargesChannelsPositive, aux )
        end
    end
    Z = zeros( Int, nChs );
    Z[ PosibleDischargesChannels ] = length.( PosibleDischargesChannelsPositive );
    aux = RemoveInfs( log.( Z ) );
    P2 = Zplot( aux, "W", :bluesreds );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

In [ ]:
if m == 6
    EventsStat = [ ];
    for i in channels
        push!( EventsStat, EventsXThrs( data[ i, : ], thr, parameters ) );
    end
    F = length.( EventsStat );
end
if m == 7
    EventsDin = [ ];
    for i in channels
        push!( EventsDin, EventsXChannel( data[ i, : ], parameters ) );
    end
    G = length.( EventsDin );
end
# auxI = unique( vcat( aux... ) );
# auxV = signal[ auxI ];
# auxIV = hcat( auxI, auxV );
# stokes = ms2Frs( Variables, Parameters[ "stokes" ] );
# using Combinatorics
# combinaciones = collect( combinations( auxI, 2 ) );
# valid = [ ];
# check = [ ];
# for i = 1:length( combinaciones )
#     aux = sort( combinaciones[ i ] );
#     if aux[ 2 ] - aux[ 1 ] > stokes
#         push!( valid, i )
#     else
#         push!( check, i )
#     end
# end
# CheckIndex = combinaciones[ check ]
# a = 1
# t = findall( auxI .∈ [ CheckIndex[ a ] ] );
# x = unique( vcat( aux... ) );
# stokes = ms2Frs( Variables, Parameters[ "stokes" ] );
# findall( diff( x ) .<= stokes )
# # using Plots
# # plot( signal )
# # vline!( [x] )
# # xlims!( 500, 2000)
# # STxSeg
# using Plots
# P1 = plot( BINRAW[50,:]);
# P2 = plot( BINRAW[50,:]);
# P2 = vline!([indexesS[50]]);
# P3 = plot( BINRAW[50,:]);
# P3 = vline!([indexesD[50]]);
# plot( P1, P2, P3, wsize = ( 800, 600 ), layout = ( 3, 1 ) )

In [ ]:
using Combinatorics
T = "S"
window = ms2Frs( Variables, Parameters[ "window" ] );
stokes = ms2Frs( Variables, Parameters[ "stokes" ] );
bit = ms2Frs( Variables, Parameters[ "bit" ] );
elements = [ ];
if T == "S"
    thr = Parameters[ "thr" ];
    elements = findall( signal .<= thr );
elseif T == "D"
    Is = collect( 1:bit:length( signal ) );
    aux = [ ];
    for I in Is
        E = I + window - 1;
        if E > length( signal )
            E = length( signal );
        end
        segment = signal[ I : E ];
        thr = -1 * Donoho( segment ) * SigmaData( segment );
        push!( aux, findall( segment .<= thr ) .+ ( I - 1 ) );
    end
    elements = sort( unique( vcat( aux... ) ) );
end
nE0 = length( elements );
nEn = 0;
while nE0 != nEn
    nE0 = length( elements );
    combinaciones = collect( combinations( elements, 2 ) );
    check = [ ];
    for i = 1:length( combinaciones )
        aux = sort( combinaciones[ i ] );
        if aux[ 2 ] - aux[ 1 ] < stokes
            push!( check, i )
        end
    end
    if !isempty( check )
        CheckIndex = combinaciones[ check ];
        y = hcat( CheckIndex... );
        x = signal[ y ];
        _, B = findmax( x, dims = 2 );
        fordelete = y[ y .∈ [ y[ B ] ] ];
        elements = elements[ elements .∉ [ fordelete ] ];
    end
    nEn = length( elements );
end

nChs = Variables[ "nChs" ];
channels = 1:nChs;

indexesS = [ ];
@time for ch = 1:nChs
    signal = BINRAW[ ch, : ];
    index = SxCh( Variables, Parameters, signal, "S" );
    push!( indexesS, index )
end

indexesD = [ ];
@time for ch = 1:nChs
    signal = BINRAW[ ch, : ];
    index = SxCh( Variables, Parameters, signal, "D" );
    push!( indexesD, index )
end

In [ ]:
# filtrando el histograma

# using LaTeXStrings
# L"h(x)=f\ast g(x)=\underset{-\infty }{\overset{\infty }{\int }}f\left(x-\tau \right)g\left(\tau \right) d\tau"
# L"g(x)=\frac{1}{\sigma \sqrt{2\pi }}{e}^{-\frac{x^2}{2{\sigma}^2}}"

using Plots
using DSP
using Statistics

sigma = 3;
cte = ( 1 / ( sigma * sqrt( 2 * pi ) ) );
potencia( x ) = -( ( x ^ 2 ) / ( 2 * ( sigma ^ 2 ) ) );
gx( x ) = cte * exp( potencia( x ) );
h = copy( RowCount );
g = gx.( h );

aux01 = h .* g;
aux02 = conv( h, g );
plot( 
    histogram( RowCount ), 
    histogram( abs.( log.( aux01 ) ) ), 
    histogram( aux02 ), 
    layout = ( 3, 1 ), 
    wsize = ( 800, 400 ),
    leg = :none
    )

l1 = maximum( RowCount ); l2 = minimum( RowCount );

data = RowCount;
P1 = histogram( data );
P2 = Zplot( data, "W", :bluesreds );
plot( 
    P1, P2, 
    leg = :none, 
    wsize = ( 800, 400 ), 
    clims = ( l2, l1 ) 
)

data = Float64.( RowCount );
σ01 = sqrt( 2 * log( length( data ) ) );
thr01 = mean( data ) + σ01 * std( data ); 
thr02 = mean( data ) - σ01 * std( data ); 
New = copy( data );
New[ New .>= thr01 ] .= thr01; 
New[ New .<= thr02 ] .= thr02;
data = New;
P1 = histogram( data );
P2 = Zplot( data, "W", :bluesreds );
plot( 
    P1, P2, 
    leg = :none, 
    wsize = ( 800, 400 ), 
    clims = ( l2, l1 ) 
)

aux01 = h .* g;
data = abs.( log.( aux01 ) );
P1 = histogram( data );
P2 = Zplot( data, "W", :bluesreds );
plot( 
    P1, P2, 
    leg = :none, 
    wsize = ( 800, 400 ), 
    #clims = ( l2, l1 ) 
)

aux01 = h .* g;
data = abs.( log.( aux01 ) );
data = RemoveInfs( data );
P1 = histogram( data );
P2 = Zplot( data, "W", :bluesreds );
plot( 
    P1, P2, 
    leg = :none, 
    wsize = ( 800, 400 ), 
    #clims = ( l2, l1 ) 
)

aux01 = h .* g;
data = abs.( log.( aux01 ) );
data = RemoveInfs( data );
σ01 = sqrt( 2 * log( length( data ) ) );
σ01 = 2;
thr01 = mean( data ) + ( σ01 * std( data ) ); 
thr02 = mean( data ) - ( σ01 * std( data ) ); 
New = copy( data );
New[ New .>= thr01 ] .= thr01; 
New[ New .<= thr02 ] .= thr02;
data = New;
P1 = histogram( data );
P2 = Zplot( data, "W", :bluesreds );
plot( 
    P1, P2, 
    leg = :none, 
    wsize = ( 800, 400 ), 
    #clims = ( l2, l1 ) 
)

using HistogramThresholding

Otsu
MinimumIntermodes
Intermodes
MinimumError
Moments
UnimodalRosin
Entropy
Balanced
Yen

aux01 = h .* g;
data = abs.( log.( aux01 ) );
data = RemoveInfs( data );
thr01 = find_threshold( data, nbins = length( unique( data ) ), Otsu( ) );
New = copy( data );
New[ New .>= thr01 ] .= 1; 
New[ New .!= 1 ] .= 0;
data = New;
P3 = Zplot( data, "W", :greys );
plot( P2, P3,
    leg = :none,
    cbar = :none,
    wsize = ( 800, 400 )
)

savefig("Yen.png")

# Otsu parece notar solo los mas activos de todos...
# MinimumIntermodes detecta los mas mas mas activos...
# Moments es parecido a Otsu pero un poco mas...
# UnimodalRosin detecta toda la rebanada y algo mas...

In [ ]:
"[------------------------------------------------------------------------------------------]"

First round of detection.

Takes about 10 minutes per 1 minute experiment.

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
limupper = 0.6; # GB maximum size of each segment considering Float16 format
ΔT = 250; # ms para el ΔV
sigma = 3; # para la convolucion gaussiana
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Constants = Dict(
    # for detecting saturations
    "maxvolt" => 3000, # μV
    "minvolt" => -3000, # μV
    # for the neighborhood
    "minchannels" => 8, # needed valid channels on the neighborhood
    "maxradius" => 4, # maximus radius for the searching neighborhood ( 1 -> 8)
    # how much of the channel can we tolerate being saturated before patching it all
    "limite_saturacion" => 0.4, 
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
using JLD
using Plots
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@suppress begin
    # PATHFunctions = "/home/isabel/Dropbox/git-repos/All-Process";
    PATHFunctions = raw"C:\Users\Siby\Dropbox\git-repos\All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# FILEBRW = "/run/media/isabel/Data/Medula/28-02-2022/BRWs/Phase_01.brw";
FILEBRW = raw"C:\Users\Siby\Desktop\Datos\Medula\28-02-2022\BRWs\Phase_01.brw";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Variables, FILEPATHS, FILEVARS = GetVarsHDF5( FILEBRW );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
σ = ChunkSizeSpace( Variables, limupper );
n0s = length( string( σ ) );
nChs = Variables[ "nChs" ];
channels = collect( 1:nChs );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS = load( FILEPATHS )[ "PATHS" ];
PATHMAIN = PATHS[ "PATHMAIN" ];
PATHFIGURES = joinpath( PATHMAIN, "Figures" ); mkpath( PATHFIGURES );
PATHFIGS_TEMP = joinpath( PATHFIGURES, "Step0" ); mkpath( PATHFIGS_TEMP );
PATHVOLTAGE = joinpath( PATHMAIN, "Voltage" ); mkpath( PATHVOLTAGE );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS[ "PATHFIGURES" ] = PATHFIGURES;
PATHS[ "PATHVOLTAGE" ] = PATHVOLTAGE;
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
save( FILEPATHS, "PATHS", PATHS );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
for n = 1:σ;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW = OneSegment( Variables, n, σ ); # bin in uInt format
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW = Digital2Analogue( Variables, BINRAW ); # bin in μV
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINNAME = string( "BIN", lpad( n, n0s, "0" ), ".jld" );
    FILEBIN = joinpath( PATHVOLTAGE, BINNAME );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    # Pre eliminacion de los extremos
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    σΔV = stdΔV( Variables, BINRAW, ΔT );
    RowCount = UniqueCount( BINRAW );
    data = RemoveInfs( log.( σΔV ) ); 
    P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
    h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
    data = RowCount; 
    P3 = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); P3 = title!( "# V values" )
    h = copy( RowCount ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
    PF1 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
    Complement = Dict( );
    Complement[ "NVoltValuesCS" ] = RowCount;
    Complement[ "stdΔVCS" ] = σΔV;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW, Discarded = RemoveSaturationExtrema( Variables, Constants, BINRAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    # Post eliminacion de los extremos
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    σΔV = stdΔV( Variables, BINRAW, ΔT );
    RowCount = UniqueCount( BINRAW );
    data = RemoveInfs( log.( σΔV ) ); 
    P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
    h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
    data = RowCount; 
    P3 = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); P3 = title!( "# V values" )
    h = copy( RowCount ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
    PF2 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
    Complement[ "NVoltValuesNS" ] = RowCount;
    Complement[ "stdΔVNS" ] = σΔV;
    Complement[ "Discarded" ] = Discarded;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    FFS0 = plot( PF1, PF2, layout = ( 2, 1 ) );
    FIGNAME = joinpath( PATHFIGS_TEMP, BINNAME );
    FIGNAME = replace( FIGNAME, ".jld" => ".svg" );
    savefig( FFS0, FIGNAME ) 
    RAWNAME = joinpath( PATHVOLTAGE, BINNAME );
    save( RAWNAME, "RAW", Float16.( BINRAW ), "Complement", Complement );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

Second round of detection.

If you have saturations for equipment or other errors, here we get rid of them.

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
using JLD
using StatsBase
using Plots
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@suppress begin
    PATHFunctions = "/home/isabel/Dropbox/git-repos/All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILEBRW = "/run/media/isabel/Data/Medula/28-02-2022/BRWs/Phase_01.brw";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILESVOLTAGE = FindContent( "Voltage", FILEBRW );
PATHINFO =  dirname( FindContent( "Info", FILEBRW )[ 1 ] );
FILESTEP0Ws = joinpath( PATHINFO, "Step0Ws.jld" );
FILESRAW = FindContent( "RAW", FILEBRW );
FILEVARIABLES = FindContent( "Variables.jld", FILEBRW )[ 1 ];
Variables = load( FILEVARIABLES )[ "Variables" ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# NVR -> Number of voltage values, Raw data
# NVP -> Number of voltage values, Repaired data
# DSR -> std from delta V of Delta T, Raw data
# DSP -> std from delta V of Delta T, Repaired data
σ = length( FILESVOLTAGE );
Empties = Array{ Any }( undef, σ );
Frames = Array{ Any }( undef, σ );
Channels = Array{ Any }( undef, σ );
NVRs = Array{ Any }( undef, σ );
NVPs = Array{ Any }( undef, σ );
DSRs = Array{ Any }( undef, σ );
DSPs = Array{ Any }( undef, σ );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@time for n = 1:σ;
    aux = load( FILESVOLTAGE[ n ] );
    Complement = aux[ "Complement" ];
    BINRAW = aux[ "RAW" ];
    Empties[ n ] = vcat( Complement[ "Discarded" ][ "Empties" ]... );
    Frames[ n ] = vcat( Complement[ "Discarded" ][ "Frames" ]... );
    Channels[ n ] = vcat( Complement[ "Discarded" ][ "Channels" ]... );
    NVRs[ n ] = Complement[ "NVoltValuesCS" ];
    NVPs[ n ] = Complement[ "NVoltValuesNS" ];
    DSRs[ n ] = Complement[ "stdΔVNS" ];
    DSPs[ n ] = Complement[ "stdΔVCS" ];
    BINNAME = replace( FILESVOLTAGE[ n ], ".jld" => "_RAW.jld" );
    save( BINNAME, "BINRAW", BINRAW );
end
Step0Ws = Dict(
    "Empties"  => Empties,
    "Frames"   => Frames,
    "Channels" => Channels,
    "NVRs"     => NVRs,
    "NVPs"     => NVPs,
    "DSRs"     => DSRs,
    "DSPs"     => DSPs
);
save( FILESTEP0Ws, "Step0Ws", Step0Ws );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

"[------------------------------------------------------------------------------------------]"

First round of detection.

Takes about 10 minutes per 1 minute experiment.

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
limupper = 0.6; # GB maximum size of each segment considering Float16 format
ΔT = 250; # ms para el ΔV
sigma = 3; # para la convolucion gaussiana
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Constants = Dict(
    # for detecting saturations
    "maxvolt" => 3000, # μV
    "minvolt" => -3000, # μV
    # for the neighborhood
    "minchannels" => 8, # needed valid channels on the neighborhood
    "maxradius" => 4, # maximus radius for the searching neighborhood ( 1 -> 8 )
    # how much of the channel can we tolerate being saturated before patching it all
    "limite_saturacion" => 0.4, 
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
using JLD
using Plots
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@suppress begin
    # PATHFunctions = "/home/isabel/Dropbox/git-repos/All-Process";
    PATHFunctions = raw"C:\Users\Siby\Dropbox\git-repos\All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# FILEBRW = "/run/media/isabel/Data/Medula/28-02-2022/BRWs/Phase_01.brw";
FILEBRW = raw"C:\Users\Siby\Desktop\Datos\Medula\28-02-2022\BRWs\Phase_01.brw";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Variables, FILEPATHS, FILEVARS = GetVarsHDF5( FILEBRW );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
σ = ChunkSizeSpace( Variables, limupper );
n0s = length( string( σ ) );
nChs = Variables[ "nChs" ];
channels = collect( 1:nChs );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS = load( FILEPATHS )[ "PATHS" ];
PATHMAIN = PATHS[ "PATHMAIN" ];
PATHFIGURES = joinpath( PATHMAIN, "Figures" ); mkpath( PATHFIGURES );
PATHFIGS_TEMP = joinpath( PATHFIGURES, "Step0" ); mkpath( PATHFIGS_TEMP );
PATHVOLTAGE = joinpath( PATHMAIN, "Voltage" ); mkpath( PATHVOLTAGE );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS[ "PATHFIGURES" ] = PATHFIGURES;
PATHS[ "PATHVOLTAGE" ] = PATHVOLTAGE;
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
save( FILEPATHS, "PATHS", PATHS );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
for n = 2:σ;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW = OneSegment( Variables, n, σ ); # bin in uInt format
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW = Digital2Analogue( Variables, BINRAW ); # bin in μV
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINNAME = string( "BIN", lpad( n, n0s, "0" ), ".jld" );
    FILEBIN = joinpath( PATHVOLTAGE, BINNAME );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    # Pre eliminacion de los extremos
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    σΔV = stdΔV( Variables, BINRAW, ΔT );
    RowCount = UniqueCount( BINRAW );
    data = RemoveInfs( log.( σΔV ) ); 
    P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
    h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
    data = RowCount; 
    P3 = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); P3 = title!( "# V values" )
    h = copy( RowCount ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
    PF1 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
    Complement = Dict( );
    Complement[ "NVoltValuesCS" ] = RowCount;
    Complement[ "stdΔVCS" ] = σΔV;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW, Discarded = RemoveSaturationExtrema( Variables, Constants, BINRAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    # Post eliminacion de los extremos
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    σΔV = stdΔV( Variables, BINRAW, ΔT );
    RowCount = UniqueCount( BINRAW );
    data = RemoveInfs( log.( σΔV ) ); 
    P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
    h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
    data = RowCount; 
    P3 = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); P3 = title!( "# V values" )
    h = copy( RowCount ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
    PF2 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
    Complement[ "NVoltValuesNS" ] = RowCount;
    Complement[ "stdΔVNS" ] = σΔV;
    Complement[ "Discarded" ] = Discarded;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    FFS0 = plot( PF1, PF2, layout = ( 2, 1 ) );
    FIGNAME = joinpath( PATHFIGS_TEMP, BINNAME );
    FIGNAME = replace( FIGNAME, ".jld" => ".svg" );
    savefig( FFS0, FIGNAME ) 
    RAWNAME = joinpath( PATHVOLTAGE, BINNAME );
    save( RAWNAME, "RAW", Float16.( BINRAW ), "Complement", Complement );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

Second round of detection.

If you have saturations for equipment or other errors, here we get rid of them.

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
using JLD
using Plots
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@suppress begin
    PATHFunctions = "/home/isabel/Dropbox/git-repos/All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILEBRW = "/run/media/isabel/Data/Medula/28-02-2022/BRWs/Phase_01.brw";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

FILESVOLTAGE = FindContent( "Voltage", FILEBRW );
PATHINFO =  dirname( FindContent( "Info", FILEBRW )[ 1 ] );
FILESTEP0Ws = joinpath( PATHINFO, "Step0Ws.jld" );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# NVR -> Number of voltage values, Raw data
# NVP -> Number of voltage values, Repaired data
# DSR -> std from delta V of Delta T, Raw data
# DSP -> std from delta V of Delta T, Repaired data
σ = length( FILESVOLTAGE );
Empties = Array{ Any }( undef, σ );
Frames = Array{ Any }( undef, σ );
Channels = Array{ Any }( undef, σ );
NVRs = Array{ Any }( undef, σ );
NVPs = Array{ Any }( undef, σ );
DSRs = Array{ Any }( undef, σ );
DSPs = Array{ Any }( undef, σ );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@time for n = 1:σ;
    aux = load( FILESVOLTAGE[ n ] );
    Complement = aux[ "Complement" ];
    BINRAW = aux[ "RAW" ];
    Empties[ n ] = vcat( Complement[ "Discarded" ][ "Empties" ]... );
    Frames[ n ] = vcat( Complement[ "Discarded" ][ "Frames" ]... );
    Channels[ n ] = vcat( Complement[ "Discarded" ][ "Channels" ]... );
    NVRs[ n ] = Complement[ "NVoltValuesCS" ];
    NVPs[ n ] = Complement[ "NVoltValuesNS" ];
    DSRs[ n ] = Complement[ "stdΔVNS" ];
    DSPs[ n ] = Complement[ "stdΔVCS" ];
    BINNAME = replace( FILESVOLTAGE[ n ], ".jld" => "_RAW.jld" );
    save( BINNAME, "BINRAW", BINRAW );
end
Step0Ws = Dict(
    "Empties"  => Empties,
    "Frames"   => Frames,
    "Channels" => Channels,
    "NVRs"     => NVRs,
    "NVPs"     => NVPs,
    "DSRs"     => DSRs,
    "DSPs"     => DSPs
);
save( FILESTEP0Ws, "Step0Ws", Step0Ws );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# FILEVARIABLES = FindContent( "Variables.jld", FILEBRW )[ 1 ];
# Variables = load( FILEVARIABLES )[ "Variables" ];

# 1. Number of voltage values present on each channel
# 2. Direct standard deviation
# 3. Standard deviation of voltage change with Δt variable
# 4. Global threshold
# 5. Standard Error
# 6. Static Threshold Events
# 7. Dynamic Threshold Events
# 8. Medians
# 9. Means

# # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# BINRAW = Float64.( BINRAW );
# lF, HF, fac = ( 290, 5010, 10 ); # MUA
# data = round.(
#     MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
# # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# NWs = 9;
# lF, HF, fac = ( 2, 302, 2 );  # LFP
# BINLFP = round.(
#     MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
#  # Rounding for less variability
# BINRAW = round.( BINRAW, digits = 3 );
# # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

"""
    AllWs( Variables::Dict{Any, Any}, BIN::Matrix{Float64}, parameters::Dict{String, Int64}, thr::Real )
        -> Ws::Matrix{Float64}, AE::Vector{Vector{Any}}, BINMUA::Matrix{Float16},
        BINLFP::Matrix{Float16}, metrics::String
        using Ms2Frs, MatrixFilter, VoltageCount, STDΔV, Donoho, EventsXThrs, EventsXChannel
        using Statistics, BinningAnalysis, DSP
        # 1. Number of voltage values present on each channel
        # 2. Direct standard deviation
        # 3. Standard deviation of voltage change with Δt variable
        # 4. Global threshold
        # 5. Standard Error
        # 6. Static Threshold Events
        # 7. Dynamic Threshold Events
        # 8. Medians
        # 9. Means
"""
function AllWs( Variables::Dict{Any, Any}, BINRAW::Matrix{Float64}, parameters::Dict{String, Int64}, thr::Real )
    # Filtering the Raw Data
    lF, HF, fac = ( 290, 5010, 10 ); # MUA
    BINMUA = round.(
        MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
    lF, HF, fac = ( 2, 302, 2 );  # LFP
    BINLFP = round.(
        MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
     # Rounding for less variability
    BINRAW = round.( BINRAW, digits = 3 );
    #
    nChs, nFrs = size( BINRAW );
    channels = collect( 1:nChs );
    Ws = zeros( Float64, nChs, 27 );
    # 1. Number of voltage values present on each channel
    Ws[ : , 1 ] = VoltageCount( BINRAW );
    Ws[ : , 2 ] = VoltageCount( BINMUA );
    Ws[ : , 3 ] = VoltageCount( BINLFP );
    # 2. Direct standard deviation
    Ws[ :, 4 ] = std( BINRAW, dims = 2 );
    Ws[ :, 5 ] = std( BINMUA, dims = 2 );
    Ws[ :, 6 ] = std( BINLFP, dims = 2 );
    # 3. Standard deviation of voltage change with Δt variable
    ΔT = parameters[ "ΔT" ];
    Ws[ :, 7 ] = STDΔV( BINRAW, ΔT );
    Ws[ :, 8 ] = STDΔV( BINMUA, ΔT );
    Ws[ :, 9 ] = STDΔV( BINLFP, ΔT );
    # 4. Global threshold ( Donoho rule )
    [ Ws[ k , 13 ] = Donoho( BINRAW[ k, : ] ) for k in channels ];
    [ Ws[ k , 14 ] = Donoho( BINMUA[ k, : ] ) for k in channels ];
    [ Ws[ k , 15 ] = Donoho( BINLFP[ k, : ] ) for k in channels ];
    # 9. Means and 5. Standard error
    for k in channels
        Ws[ k, 25 ], Ws[ k, 10 ] = jackknife( identity, BINRAW[ k, : ] );
        Ws[ k, 26 ], Ws[ k, 11 ] = jackknife( identity, BINMUA[ k, : ] );
        Ws[ k, 27 ], Ws[ k, 12 ] = jackknife( identity, BINLFP[ k, : ] );
    end
    # 6. Static Threshold Events and 7. Dynamic Threshold Events
    EventsRAWStat = [ ]; EventsMUAStat = [ ]; EventsLFPStat = [ ];
    for i in channels
        push!( EventsRAWStat, EventsXThrs( BINRAW[ i, : ], thr, parameters ) );
        push!( EventsMUAStat, EventsXThrs( BINMUA[ i, : ], thr, parameters ) );
        push!( EventsLFPStat, EventsXThrs( BINLFP[ i, : ], thr, parameters ) );
    end
    EventsMUADin = [ ]; EventsRAWDin = [ ]; EventsLFPDin = [ ];
    for i in channels
        _, IndexRAW = EventsXChannel( BINRAW[ i, : ], parameters );
        _, IndexMUA = EventsXChannel( BINMUA[ i, : ], parameters );
        _, IndexLFP = EventsXChannel( BINLFP[ i, : ], parameters );
        push!( EventsRAWDin, IndexRAW );
        push!( EventsMUADin, IndexMUA );
        push!( EventsLFPDin, IndexLFP );
    end
    Ws[ :, 16 ] = length.( EventsRAWStat );
    Ws[ :, 17 ] = length.( EventsMUAStat );
    Ws[ :, 18 ] = length.( EventsLFPStat );
    Ws[ :, 19 ] = length.( EventsRAWDin );
    Ws[ :, 20 ] = length.( EventsMUADin );
    Ws[ :, 21 ] = length.( EventsLFPDin );
    # 8. Medians
    Ws[ :, 22 ] = median( BINRAW, dims = 2 );
    Ws[ :, 23 ] = median( BINMUA, dims = 2 );
    Ws[ :, 24 ] = median( BINLFP, dims = 2 );
    AE = [ EventsRAWStat, EventsRAWDin, EventsMUAStat, EventsMUADin, EventsLFPStat,
        EventsLFPDin ];
    metrics = [
        "# Voltage, log, N, RAW", "# Voltage, log, N, MUA", "# Voltage, log, N, LFP",
        "std, log, N, RAW", "std, log, N, MUA", "std, log, N, LFP",
        "std Δt, log, N, RAW", "std Δt, log, N, MUA", "std Δt, log, N, LFP",
        "std-error, log, N, RAW", "std-error, log, N, MUA", "std-error, log, N, LFP",
        "threshold, log, N, RAW", "threshold, log, N, MUA", "threshold, log, N, LFP",
        "EventsStat, log, N, RAW", "EventsStat, log, N, MUA", "EventsStat, log, N, LFP",
        "EventsDin, log, N, RAW", "EventsDin, log, N, MUA", "EventsDin, log, N, LFP",
        "Median, log, N, RAW", "Median, log, N, MUA", "Median, log, N, LFP",
        "Mean, log, N, RAW", "Mean, log, N, MUA", "Mean, log, N, LFP",
    ];
    return Ws, AE, Float16.( BINMUA ), Float16.( BINLFP ), metrics
end

"[------------------------------------------------------------------------------------------]"

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
limupper = 0.6; # GB maximum size of each segment considering Float16 format
ΔT = 250; # ms para el ΔV
sigma = 3; # para la convolucion gaussiana
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Constants = Dict(
    # for detecting saturations
    "maxvolt" => 3000, # μV
    "minvolt" => -3000, # μV
    # for the neighborhood
    "maxrange" => 500, # μV 
    "minrange" => -1000, # μV
    "minchannels" => 4, # valid channels on the neighborhood
    "maxradius" => 3, # maximus radious for searching the neighborhood
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
using JLD
using Plots
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@suppress begin
    PATHFunctions = 
    "/home/isabel/Dropbox/git-repos/All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILEBRW = "/run/media/isabel/Data/Medula/28-02-2022/BRWs/Phase_01.brw";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Variables, FILEPATHS, FILEVARS = GetVarsHDF5( FILEBRW );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
σ = ChunkSizeSpace( Variables, limupper );
n0s = length( string( σ ) );
nChs = Variables[ "nChs" ];
channels = collect( 1:nChs );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS = load( FILEPATHS )[ "PATHS" ];
PATHMAIN = PATHS[ "PATHMAIN" ];
PATHFIGURES = joinpath( PATHMAIN, "Figures" ); mkpath( PATHFIGURES );
PATHFIGS_TEMP = joinpath( PATHFIGURES, "Step0" ); mkpath( PATHFIGS_TEMP );
PATHVOLTAGE = joinpath( PATHMAIN, "Voltage" ); mkpath( PATHVOLTAGE );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS[ "PATHFIGURES" ] = PATHFIGURES;
PATHS[ "PATHVOLTAGE" ] = PATHVOLTAGE;
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
save( FILEPATHS, "PATHS", PATHS );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
n = 1
BIN = OneSegment( Variables, n, σ ); # bin in uInt format
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
BIN = Digital2Analogue( Variables, BIN ); # bin in μV
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
BINNAME = string( "BIN", lpad( n, n0s, "0" ), ".jld" );
FILEBIN = joinpath( PATHVOLTAGE, BINNAME );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# Pre eliminacion de los extremos
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
σΔV = stdΔV( Variables, BIN, ΔT );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
RowCount = UniqueCount( BIN );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
data = RemoveInfs( log.( σΔV ) ); 
P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
data = RowCount; 
P3 = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); P3 = title!( "# V values" )
h = copy( RowCount ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
PF1 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Discarted, BIN = RemoveExtrema( Variables, Constants, BIN );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# Post eliminacion de los extremos
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
σΔV = stdΔV( Variables, BIN, ΔT );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
RowCount = UniqueCount( BIN );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
data = RemoveInfs( log.( σΔV ) ); 
P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
data = RowCount; 
P3 = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); P3 = title!( "# V values" )
h = copy( RowCount ); hg = convgauss( sigma, h );
data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
PF2 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

"[------------------------------------------------------------------------------------------]"

First round of detection.

Takes about 10 minutes per 1 minute experiment.

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~ constantes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ #
limupper = 0.6; # GB maximum size of each segment considering Float16 format
ΔT = 250; # ms para el ΔV
sigma = 3; # para la convolucion gaussiana
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Constants = Dict(
    # for detecting saturations
    "maxvolt" => 3000, # μV
    "minvolt" => -3000, # μV
    # for the neighborhood
    "minchannels" => 8, # needed valid channels on the neighborhood
    "maxradius" => 4, # maximus radius for the searching neighborhood ( 1 -> 8)
    # how much of the channel can we tolerate being saturated before patching it all
    "limite_saturacion" => 0.4, 
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
using JLD
using Plots
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@suppress begin
    # PATHFunctions = "/home/isabel/Dropbox/git-repos/All-Process";
    PATHFunctions = raw"C:\Users\Siby\Dropbox\git-repos\All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# FILEBRW = "/run/media/isabel/Data/Medula/28-02-2022/BRWs/Phase_01.brw";
FILEBRW = raw"C:\Users\Siby\Desktop\Datos\Medula\28-02-2022\BRWs\Phase_01.brw";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Variables, FILEPATHS, FILEVARS = GetVarsHDF5( FILEBRW );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
σ = ChunkSizeSpace( Variables, limupper );
n0s = length( string( σ ) );
nChs = Variables[ "nChs" ];
channels = collect( 1:nChs );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS = load( FILEPATHS )[ "PATHS" ];
PATHMAIN = PATHS[ "PATHMAIN" ];
PATHFIGURES = joinpath( PATHMAIN, "Figures" ); mkpath( PATHFIGURES );
PATHFIGS_TEMP = joinpath( PATHFIGURES, "Step0" ); mkpath( PATHFIGS_TEMP );
PATHVOLTAGE = joinpath( PATHMAIN, "Voltage" ); mkpath( PATHVOLTAGE );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
PATHS[ "PATHFIGURES" ] = PATHFIGURES;
PATHS[ "PATHVOLTAGE" ] = PATHVOLTAGE;
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
save( FILEPATHS, "PATHS", PATHS );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
for n = 1:σ;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW = OneSegment( Variables, n, σ ); # bin in uInt format
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW = Digital2Analogue( Variables, BINRAW ); # bin in μV
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINNAME = string( "BIN", lpad( n, n0s, "0" ), ".jld" );
    FILEBIN = joinpath( PATHVOLTAGE, BINNAME );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    # Pre eliminacion de los extremos
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    σΔV = stdΔV( Variables, BINRAW, ΔT );
    RowCount = UniqueCount( BINRAW );
    data = RemoveInfs( log.( σΔV ) ); 
    P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
    h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
    data = RowCount; 
    P3 = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); P3 = title!( "# V values" )
    h = copy( RowCount ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
    PF1 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
    Complement = Dict( );
    Complement[ "NVoltValuesCS" ] = RowCount;
    Complement[ "stdΔVCS" ] = σΔV;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    BINRAW, Discarded = RemoveSaturationExtrema( Variables, Constants, BINRAW );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    # Post eliminacion de los extremos
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    σΔV = stdΔV( Variables, BINRAW, ΔT );
    RowCount = UniqueCount( BINRAW );
    data = RemoveInfs( log.( σΔV ) ); 
    P1 = Zplot( data, "W", :bluesreds ); P1 = title!( "std ΔV" );
    h = vec( copy( σΔV ) ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P2 = Zplot( data, "W", :bluesreds ); P2 = title!( "std ΔV conv" );
    data = RowCount; 
    P3 = Zplot( RemoveInfs( log.( data ) ), "W", :bluesreds ); P3 = title!( "# V values" )
    h = copy( RowCount ); hg = convgauss( sigma, h );
    data = vec( RemoveInfs( abs.( log.( hg ) ) ) ); 
    P4 = Zplot( data, "W", :bluesreds ); P4 = title!( "# V values conv" )
    PF2 = plot( P1, P2, P3, P4, layout = ( 1, 4 ), wsize = ( 800, 300 ), cbar = :none );
    Complement[ "NVoltValuesNS" ] = RowCount;
    Complement[ "stdΔVNS" ] = σΔV;
    Complement[ "Discarded" ] = Discarded;
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
    FFS0 = plot( PF1, PF2, layout = ( 2, 1 ) );
    FIGNAME = joinpath( PATHFIGS_TEMP, BINNAME );
    FIGNAME = replace( FIGNAME, ".jld" => ".svg" );
    savefig( FFS0, FIGNAME ) 
    RAWNAME = joinpath( PATHVOLTAGE, BINNAME );
    save( RAWNAME, "RAW", Float16.( BINRAW ), "Complement", Complement );
    # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

Second round of detection.

If you have saturations for equipment or other errors, here we get rid of them.

# ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ paquetes ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~~ #
using Suppressor
using JLD
using Plots
using StatsBase
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@suppress begin
    PATHFunctions = raw"C:\Users\Siby\Dropbox\git-repos\All-Process";
    push!( LOAD_PATH, PATHFunctions );
    using F2023
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILEBRW = raw"C:\Users\Siby\Desktop\Datos\Medula\28-02-2022\BRWs\Phase_01.brw";
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
FILESVOLTAGE = FindContent( "Voltage", FILEBRW );
PATHINFO =  dirname( FindContent( "Info", FILEBRW )[ 1 ] );
FILESTEP0Ws = joinpath( PATHINFO, "Step0Ws.jld" );
FILESRAW = FindContent( "RAW", FILEBRW );
FILEVARIABLES = FindContent( "Variables.jld", FILEBRW )[ 1 ];
Variables = load( FILEVARIABLES )[ "Variables" ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# NVR -> Number of voltage values, Raw data
# NVP -> Number of voltage values, Repaired data
# DSR -> std from delta V of Delta T, Raw data
# DSP -> std from delta V of Delta T, Repaired data
σ = length( FILESVOLTAGE );
Empties = Array{ Any }( undef, σ );
Frames = Array{ Any }( undef, σ );
Channels = Array{ Any }( undef, σ );
NVRs = Array{ Any }( undef, σ );
NVPs = Array{ Any }( undef, σ );
DSRs = Array{ Any }( undef, σ );
DSPs = Array{ Any }( undef, σ );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
@time for n = 1:σ;
    aux = load( FILESVOLTAGE[ n ] );
    Complement = aux[ "Complement" ];
    BINRAW = aux[ "RAW" ];
    Empties[ n ] = vcat( Complement[ "Discarded" ][ "Empties" ]... );
    Frames[ n ] = vcat( Complement[ "Discarded" ][ "Frames" ]... );
    Channels[ n ] = vcat( Complement[ "Discarded" ][ "Channels" ]... );
    NVRs[ n ] = Complement[ "NVoltValuesCS" ];
    NVPs[ n ] = Complement[ "NVoltValuesNS" ];
    DSRs[ n ] = Complement[ "stdΔVNS" ];
    DSPs[ n ] = Complement[ "stdΔVCS" ];
    BINNAME = replace( FILESVOLTAGE[ n ], ".jld" => "_RAW.jld" );
    save( BINNAME, "BINRAW", BINRAW );
end
Step0Ws = Dict(
    "Empties"  => Empties,
    "Frames"   => Frames,
    "Channels" => Channels,
    "NVRs"     => NVRs,
    "NVPs"     => NVPs,
    "DSRs"     => DSRs,
    "DSPs"     => DSPs
);
save( FILESTEP0Ws, "Step0Ws", Step0Ws );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
Step0Ws = load( FILESTEP0Ws )[ "Step0Ws" ];
Empties = Step0Ws[ "Empties" ];
Frames = Step0Ws[ "Frames" ]; 
Channels = Step0Ws[ "Channels" ];
NVRs = Step0Ws[ "NVRs" ];
NVPs = Step0Ws[ "NVPs" ];
DSRs = Step0Ws[ "DSRs" ];
DSPs = Step0Ws[ "DSPs" ];
aux00 = sort( unique( vcat( Empties... ) ) );
aux01 = std.( NVRs );
thr = mean( aux01 ) + 2*std( aux01 );
segments = findall( aux01 .>= thr );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
i = 1;
FILERAW = FILESRAW[ segments[ i ] ];
@time BINRAW = load( FILERAW )[ "BINRAW" ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
BINRAW = Float64.( BINRAW );
lF, HF, fac = ( 290, 5010, 10 ); # MUA
data = round.(
    MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
using StatsBase
Donoho( x ) = ( median( abs.( x ) ) / 0.6745 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
nchannels = size( data, 1 );
thrs = [ ];
for j in 1:nchannels
    push!( thrs, Donoho( data[ j, : ] ) )
end
σ0 = sqrt( 2*log( size( data, 2 ) ) );
ST = [ ];
NST = [ ];
for j in 1:nchannels
    thr = thrs[ j ]*σ0;
    ch = data[ j, : ];
    st = findall( ch .> thr );
    push!( ST, st );
    push!( NST, length( st ) );
end
aux = vcat( ST... );
K = collect( keys( countmap( aux ) ) );
V = collect( values( countmap( aux ) ) );
σ1 = sqrt( 2*log( length( V ) ) );
thr = Donoho( V );
PosibleDischarges = K[ V .>= thr*σ1 ];
PosibleDischargesChannels = [ ];
PosibleDischargesChannelsPositive = [ ];
for j = 1:nchannels
    aux = findall( ST[ j ] .∈ [ PosibleDischarges ] );
    if !isempty( aux )
        push!( PosibleDischargesChannels, j )
        push!( PosibleDischargesChannelsPositive, aux )
    end
end
Z = zeros( Int, nchannels );
Z[ PosibleDischargesChannels ] = length.(PosibleDischargesChannelsPositive);
aux = RemoveInfs( log.( Z ) );
F4 = Zplot( aux, "W", :bluesreds );
using HistogramThresholding
x = aux;
a1 = HistogramThresholding.find_threshold( x, Balanced( ) );
a2 = HistogramThresholding.find_threshold( x, Entropy( ) );
a3 = HistogramThresholding.find_threshold( x, Intermodes( ) );
a4 = HistogramThresholding.find_threshold( x, MinimumError( ) );
a5 = HistogramThresholding.find_threshold( x, MinimumIntermodes( ) );
a6 = HistogramThresholding.find_threshold( x, Moments( ) );
a7 = HistogramThresholding.find_threshold( x, Otsu( ) );
a8 = HistogramThresholding.find_threshold( x, UnimodalRosin( ) );
a9 = HistogramThresholding.find_threshold( x, Yen( ) );
P1 = Zplot( findall( x .>= a1 ),"0" ); 
y = length(findall( x .>= a1 ) ); P1 = title!( "chs: $y" );
P2 = Zplot( findall( x .>= a2 ),"0" ); 
y = length(findall( x .>= a2 ) ); P2 = title!( "chs: $y" );
P3 = Zplot( findall( x .>= a3 ),"0" ); 
y = length(findall( x .>= a3 ) ); P3 = title!( "chs: $y" );
P4 = Zplot( findall( x .>= a4 ),"0" ); 
y = length(findall( x .>= a4 ) ); P4 = title!( "chs: $y" );
P5 = Zplot( findall( x .>= a5 ),"0" ); 
y = length(findall( x .>= a5 ) ); P5 = title!( "chs: $y" );
P6 = Zplot( findall( x .>= a6 ),"0" ); 
y = length(findall( x .>= a6 ) ); P6 = title!( "chs: $y" );
P7 = Zplot( findall( x .>= a7 ),"0" ); 
y = length(findall( x .>= a7 ) ); P7 = title!( "chs: $y" );
P8 = Zplot( findall( x .>= a8 ),"0" ); 
y = length(findall( x .>= a8 ) ); P8 = title!( "chs: $y" );
P9 = Zplot( findall( x .>= a9 ),"0" ); 
y = length( findall( x .>= a9 ) ); P9 = title!( "chs: $y" );
σ0 = sqrt( 2*log( length( x ) ) );
a10 = Donoho( x )*σ0;
P10 = Zplot( findall( x .>= a10 ),"0" ); 
y = length( findall( x .>= a10 ) ); P10 = title!( "chs: $y" );

PF = plot( 
    P1, P2, P3, P4, P5, P6, P7, P8, P9, P10,
    layout = ( 4, 3 ), wsize = ( 800, 800 ), cbar = :none
);
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

T = [ a1, a2, a3, a4, a5, a6, a7, a8, a9, a10 ];
T = unique( T );
T = T[ T .!= 0 ];
TF = minimum( T );
Zplot(findall( x .> TF ), "0")
    
# 1. Number of voltage values present on each channel
# 2. Direct standard deviation
# 3. Standard deviation of voltage change with Δt variable
# 4. Global threshold
# 5. Standard Error
# 6. Static Threshold Events
# 7. Dynamic Threshold Events
# 8. Medians
# 9. Means
    
# NWs = 9;
# lF, HF, fac = ( 2, 302, 2 );  # LFP
# BINLFP = round.(
#     MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
#  # Rounding for less variability
# BINRAW = round.( BINRAW, digits = 3 );
# # ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
# bar( aux01, leg = :none )
# vline!( [ 10, 25, 40 ] )
# hline!( [ mean( aux01 ) + 2*std( aux01 ) ] )
# bounds = collect( 1:15:4096 );
# α = length( bounds ) - 1;
# ChsPlots = Array{ Plots.Plot{ Plots.GRBackend } }( undef, α );
# for n = 1:α
#     indexes = collect( bounds[ n ]:( bounds[ n + 1 ] ) );
#     ChsPlots[ n ] = plot( 
#         BINRAW[ indexes, : ]', 
#         layout = ( length( indexes ), 1 ), 
#         wsize = ( 800, 1600), leg = :none, 
#         ylims = ( -300, 300 ), 
#         framestyle = :none, 
#         lw = 0.2, 
#         color = :black );
# end

"""
    AllWs( Variables::Dict{Any, Any}, BIN::Matrix{Float64}, parameters::Dict{String, Int64}, thr::Real )
        -> Ws::Matrix{Float64}, AE::Vector{Vector{Any}}, BINMUA::Matrix{Float16},
        BINLFP::Matrix{Float16}, metrics::String
        using Ms2Frs, MatrixFilter, VoltageCount, STDΔV, Donoho, EventsXThrs, EventsXChannel
        using Statistics, BinningAnalysis, DSP
        # 1. Number of voltage values present on each channel
        # 2. Direct standard deviation
        # 3. Standard deviation of voltage change with Δt variable
        # 4. Global threshold
        # 5. Standard Error
        # 6. Static Threshold Events
        # 7. Dynamic Threshold Events
        # 8. Medians
        # 9. Means
"""
function AllWs( Variables::Dict{Any, Any}, BINRAW::Matrix{Float64}, parameters::Dict{String, Int64}, thr::Real )
    # Filtering the Raw Data
    lF, HF, fac = ( 290, 5010, 10 ); # MUA
    BINMUA = round.(
        MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
    lF, HF, fac = ( 2, 302, 2 );  # LFP
    BINLFP = round.(
        MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
     # Rounding for less variability
    BINRAW = round.( BINRAW, digits = 3 );
    #
    nChs, nFrs = size( BINRAW );
    channels = collect( 1:nChs );
    Ws = zeros( Float64, nChs, 27 );
    # 1. Number of voltage values present on each channel
    Ws[ : , 1 ] = VoltageCount( BINRAW );
    Ws[ : , 2 ] = VoltageCount( BINMUA );
    Ws[ : , 3 ] = VoltageCount( BINLFP );
    # 2. Direct standard deviation
    Ws[ :, 4 ] = std( BINRAW, dims = 2 );
    Ws[ :, 5 ] = std( BINMUA, dims = 2 );
    Ws[ :, 6 ] = std( BINLFP, dims = 2 );
    # 3. Standard deviation of voltage change with Δt variable
    ΔT = parameters[ "ΔT" ];
    Ws[ :, 7 ] = STDΔV( BINRAW, ΔT );
    Ws[ :, 8 ] = STDΔV( BINMUA, ΔT );
    Ws[ :, 9 ] = STDΔV( BINLFP, ΔT );
    # 4. Global threshold ( Donoho rule )
    [ Ws[ k , 13 ] = Donoho( BINRAW[ k, : ] ) for k in channels ];
    [ Ws[ k , 14 ] = Donoho( BINMUA[ k, : ] ) for k in channels ];
    [ Ws[ k , 15 ] = Donoho( BINLFP[ k, : ] ) for k in channels ];
    # 9. Means and 5. Standard error
    for k in channels
        Ws[ k, 25 ], Ws[ k, 10 ] = jackknife( identity, BINRAW[ k, : ] );
        Ws[ k, 26 ], Ws[ k, 11 ] = jackknife( identity, BINMUA[ k, : ] );
        Ws[ k, 27 ], Ws[ k, 12 ] = jackknife( identity, BINLFP[ k, : ] );
    end
    # 6. Static Threshold Events and 7. Dynamic Threshold Events
    EventsRAWStat = [ ]; EventsMUAStat = [ ]; EventsLFPStat = [ ];
    for i in channels
        push!( EventsRAWStat, EventsXThrs( BINRAW[ i, : ], thr, parameters ) );
        push!( EventsMUAStat, EventsXThrs( BINMUA[ i, : ], thr, parameters ) );
        push!( EventsLFPStat, EventsXThrs( BINLFP[ i, : ], thr, parameters ) );
    end
    EventsMUADin = [ ]; EventsRAWDin = [ ]; EventsLFPDin = [ ];
    for i in channels
        _, IndexRAW = EventsXChannel( BINRAW[ i, : ], parameters );
        _, IndexMUA = EventsXChannel( BINMUA[ i, : ], parameters );
        _, IndexLFP = EventsXChannel( BINLFP[ i, : ], parameters );
        push!( EventsRAWDin, IndexRAW );
        push!( EventsMUADin, IndexMUA );
        push!( EventsLFPDin, IndexLFP );
    end
    Ws[ :, 16 ] = length.( EventsRAWStat );
    Ws[ :, 17 ] = length.( EventsMUAStat );
    Ws[ :, 18 ] = length.( EventsLFPStat );
    Ws[ :, 19 ] = length.( EventsRAWDin );
    Ws[ :, 20 ] = length.( EventsMUADin );
    Ws[ :, 21 ] = length.( EventsLFPDin );
    # 8. Medians
    Ws[ :, 22 ] = median( BINRAW, dims = 2 );
    Ws[ :, 23 ] = median( BINMUA, dims = 2 );
    Ws[ :, 24 ] = median( BINLFP, dims = 2 );
    AE = [ EventsRAWStat, EventsRAWDin, EventsMUAStat, EventsMUADin, EventsLFPStat,
        EventsLFPDin ];
    metrics = [
        "# Voltage, log, N, RAW", "# Voltage, log, N, MUA", "# Voltage, log, N, LFP",
        "std, log, N, RAW", "std, log, N, MUA", "std, log, N, LFP",
        "std Δt, log, N, RAW", "std Δt, log, N, MUA", "std Δt, log, N, LFP",
        "std-error, log, N, RAW", "std-error, log, N, MUA", "std-error, log, N, LFP",
        "threshold, log, N, RAW", "threshold, log, N, MUA", "threshold, log, N, LFP",
        "EventsStat, log, N, RAW", "EventsStat, log, N, MUA", "EventsStat, log, N, LFP",
        "EventsDin, log, N, RAW", "EventsDin, log, N, MUA", "EventsDin, log, N, LFP",
        "Median, log, N, RAW", "Median, log, N, MUA", "Median, log, N, LFP",
        "Mean, log, N, RAW", "Mean, log, N, MUA", "Mean, log, N, LFP",
    ];
    return Ws, AE, Float16.( BINMUA ), Float16.( BINLFP ), metrics
end

In [ ]:
"""
    AllWs( Variables::Dict{Any, Any}, BIN::Matrix{Float64}, parameters::Dict{String, Int64}, thr::Real )
        -> Ws::Matrix{Float64}, AE::Vector{Vector{Any}}, BINMUA::Matrix{Float16},
        BINLFP::Matrix{Float16}, metrics::String
        using Ms2Frs, MatrixFilter, VoltageCount, STDΔV, Donoho, EventsXThrs, EventsXChannel
        using Statistics, BinningAnalysis, DSP
        # 1. Number of voltage values present on each channel
        # 2. Direct standard deviation
        # 3. Standard deviation of voltage change with Δt variable
        # 4. Global threshold
        # 5. Standard Error
        # 6. Static Threshold Events
        # 7. Dynamic Threshold Events
        # 8. Medians
        # 9. Means
"""
function AllWs( Variables::Dict{Any, Any}, BINRAW::Matrix{Float64}, 
        parameters::Dict{String, Int64}, thr::Real )
    # Filtering the Raw Data
    lF, HF, fac = ( 290, 5010, 10 ); # MUA
    BINMUA = round.(
        MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
    lF, HF, fac = ( 2, 302, 2 );  # LFP
    BINLFP = round.(
        MatrixFilter( Variables, BINRAW, lF, HF, fac, "BPF" ), digits = 3 );
     # Rounding for less variability
    BINRAW = round.( BINRAW, digits = 3 );
    #
    nChs, nFrs = size( BINRAW );
    channels = collect( 1:nChs );
    Ws = zeros( Float64, nChs, 27 );
    # 1. Number of voltage values present on each channel
    Ws[ : , 1 ] = VoltageCount( BINRAW );
    Ws[ : , 2 ] = VoltageCount( BINMUA );
    Ws[ : , 3 ] = VoltageCount( BINLFP );
    # 2. Direct standard deviation
    Ws[ :, 4 ] = std( BINRAW, dims = 2 );
    Ws[ :, 5 ] = std( BINMUA, dims = 2 );
    Ws[ :, 6 ] = std( BINLFP, dims = 2 );
    # 3. Standard deviation of voltage change with Δt variable
    ΔT = parameters[ "ΔT" ];
    Ws[ :, 7 ] = STDΔV( BINRAW, ΔT );
    Ws[ :, 8 ] = STDΔV( BINMUA, ΔT );
    Ws[ :, 9 ] = STDΔV( BINLFP, ΔT );
    # 4. Global threshold ( Donoho rule )
    [ Ws[ k , 13 ] = Donoho( BINRAW[ k, : ] ) for k in channels ];
    [ Ws[ k , 14 ] = Donoho( BINMUA[ k, : ] ) for k in channels ];
    [ Ws[ k , 15 ] = Donoho( BINLFP[ k, : ] ) for k in channels ];
    # 9. Means and 5. Standard error
    for k in channels
        Ws[ k, 25 ], Ws[ k, 10 ] = jackknife( identity, BINRAW[ k, : ] );
        Ws[ k, 26 ], Ws[ k, 11 ] = jackknife( identity, BINMUA[ k, : ] );
        Ws[ k, 27 ], Ws[ k, 12 ] = jackknife( identity, BINLFP[ k, : ] );
    end
    # 6. Static Threshold Events and 7. Dynamic Threshold Events
    EventsRAWStat = [ ]; EventsMUAStat = [ ]; EventsLFPStat = [ ];
    for i in channels
        push!( EventsRAWStat, EventsXThrs( BINRAW[ i, : ], thr, parameters ) );
        push!( EventsMUAStat, EventsXThrs( BINMUA[ i, : ], thr, parameters ) );
        push!( EventsLFPStat, EventsXThrs( BINLFP[ i, : ], thr, parameters ) );
    end
    EventsMUADin = [ ]; EventsRAWDin = [ ]; EventsLFPDin = [ ];
    for i in channels
        _, IndexRAW = EventsXChannel( BINRAW[ i, : ], parameters );
        _, IndexMUA = EventsXChannel( BINMUA[ i, : ], parameters );
        _, IndexLFP = EventsXChannel( BINLFP[ i, : ], parameters );
        push!( EventsRAWDin, IndexRAW );
        push!( EventsMUADin, IndexMUA );
        push!( EventsLFPDin, IndexLFP );
    end
    Ws[ :, 16 ] = length.( EventsRAWStat );
    Ws[ :, 17 ] = length.( EventsMUAStat );
    Ws[ :, 18 ] = length.( EventsLFPStat );
    Ws[ :, 19 ] = length.( EventsRAWDin );
    Ws[ :, 20 ] = length.( EventsMUADin );
    Ws[ :, 21 ] = length.( EventsLFPDin );
    # 8. Medians
    Ws[ :, 22 ] = median( BINRAW, dims = 2 );
    Ws[ :, 23 ] = median( BINMUA, dims = 2 );
    Ws[ :, 24 ] = median( BINLFP, dims = 2 );
    AE = [ EventsRAWStat, EventsRAWDin, EventsMUAStat, EventsMUADin, EventsLFPStat,
        EventsLFPDin ];
    metrics = [
        "# Voltage, log, N, RAW", "# Voltage, log, N, MUA", "# Voltage, log, N, LFP",
        "std, log, N, RAW", "std, log, N, MUA", "std, log, N, LFP",
        "std Δt, log, N, RAW", "std Δt, log, N, MUA", "std Δt, log, N, LFP",
        "std-error, log, N, RAW", "std-error, log, N, MUA", "std-error, log, N, LFP",
        "threshold, log, N, RAW", "threshold, log, N, MUA", "threshold, log, N, LFP",
        "EventsStat, log, N, RAW", "EventsStat, log, N, MUA", "EventsStat, log, N, LFP",
        "EventsDin, log, N, RAW", "EventsDin, log, N, MUA", "EventsDin, log, N, LFP",
        "Median, log, N, RAW", "Median, log, N, MUA", "Median, log, N, LFP",
        "Mean, log, N, RAW", "Mean, log, N, MUA", "Mean, log, N, LFP",
    ];
    return Ws, AE, Float16.( BINMUA ), Float16.( BINLFP ), metrics
end

In [ ]:
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
temporal = UniqueCount( BINPATCH );
NVoltValuesRaw2 = copy( NVoltValuesRaw );
NVoltValuesRaw2[ α ] = temporal;
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #
data = RemoveInfs( std.( NVoltValuesRaw2 ) );
thr = mean( data ) + 2*std( data );
segments = findall( data .>= thr );
P3 = bar( data, leg = :none, wsize = ( 800, 800 ) );
vline!( [ segments ] );
hline!( [ thr ] );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
ST = [ ];
thrs = [ ];
PosibleDischargesChannels = [ ];
PosibleDischargesChannelsPositive = [ ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
data = [ ];
data = round.(
    MatrixFilter( Variables, BINPATCH, lF, HF, fac, "BPF" ), digits = 3 );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
ST = [ ];
thrs = [ ]; 
PosibleDischargesChannels = [ ]; 
PosibleDischargesChannelsPositive = [ ];
for j in 1:nChs
    thr = Donoho( data[ j, : ] ) * SigmaData( data[ j, : ] );
    ch = data[ j, : ];
    st = findall( ch .> thr );
    push!( ST, st );
end
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
K = collect( keys( countmap( vcat( ST... ) ) ) );
V = collect( values( countmap( vcat( ST... ) ) ) );
PosibleDischarges = K[ V .>= Donoho( V ) * SigmaData( V ) ];
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
for j = 1:nChs
    aux = findall( ST[ j ] .∈ [ PosibleDischarges ] );
    if !isempty( aux )
        push!( PosibleDischargesChannels, j )
        push!( PosibleDischargesChannelsPositive, aux )
    end
end
Z = zeros( Int, nChs );
Z[ PosibleDischargesChannels ] = length.( PosibleDischargesChannelsPositive );
aux = RemoveInfs( log.( Z ) );
P4 = Zplot( aux, "W", :bluesreds );
cl00, clNN = extrema( aux );
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

plot( P1, P3, layout = ( 2, 1 ) )

# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #
clims = ( minimum( [ cl0, cl00 ] ), maximum( [ clN, clNN ] ) );
plot( P2, P4, layout = ( 1, 2 ), wsize = ( 800, 400 ), clims = clims )
# ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~ ~ ~~  #

5

7-element Vector{Float64}:
 -4.6
 -4.2
 -4.0
 -2.8
 -1.8
 -0.4
  0.0